
# F1 Fantasy — GP Las Vegas (2025) — **VEPI & Team Optimizer v2**
Recursos adicionais:
- **Locks** (sempre incluir) e **Exclusions** (nunca incluir) para **pilotos** e **construtores**  
- **Multiplicadores** por piloto (ex.: capitão 2×, mega 3×)  
- **Modelagem de Construtor**: pontos = **soma dos 2 pilotos da equipe** + **pit points** (faixas e bônus)  
- **1 ou 2 construtores** (parametrizável) dentro do **orçamento total**

> Observação: este otimizador não usa odds; ele combina histórico técnico (VEPI) + seus inputs Fantasy.


In [188]:

# =====================
# Parâmetros principais
# =====================

# --- Night / Urban aliases que batem com seus arquivos ---
NIGHT_TRACK_ALIASES = {
    "Las Vegas": ["Vegas"],
    "Miami": ["Miami", "Miami Gardens"],
    "Singapore": ["Singapore", "Marina Bay"],
    "Saudi": ["Jeddah", "Saudi"],
    "Abu Dhabi": ["Yas Island", "Abu Dhabi"],
    "Bahrain": ["Sakhir", "Bahrain"],
    "Qatar": ["Lusail", "Qatar"]
}

def is_night_urban(name: str) -> bool:
    """Retorna True se o nome do arquivo/GP contiver algum alias noturno/urbano."""
    if not isinstance(name, str):
        return False
    low = name.casefold()
    for aliases in NIGHT_TRACK_ALIASES.values():
        for a in aliases:
            if a.casefold() in low:
                return True
    return False


# Diretório dos race traces locais
RACE_TRACES_DIR = "C:/F1/setup/f1_race_traces_2021/f1_race_traces_2021/data/laps_2025" # ajuste se necessário

# Excel gerado anteriormente com lista de pilotos (opcional)
FANTASY_INPUT_XLSX = "C:/F1/setup/f1_race_traces_2021/F1_Fantasy_GP_Sao_Paulo_2025_3Camadas.xlsx"  # ajuste se desejar

# Saída
OUT_EXCEL = "F1_Fantasy_Vegas_Projections_v2.xlsx"

# Corridas similares a Vegas (urbana/noturna)
URBAN_NIGHT_TRACKS = {"Las Vegas", "Miami", "Singapore", "Saudi", "Abu Dhabi", "Bahrain", "Qatar"}

# Sprint weekend de Vegas?
SPRINT_WEEKEND = False
SPRINT_UPLIFT = 1.25

# Orçamento total (pilotos + construtor(es))
TOTAL_BUDGET = 119.6

# Quantidade de construtores (1 padrão; pode usar 2 se seu jogo permitir)
NUM_CONSTRUCTORS = 2  # mude para 2 se quiser

# Locks/Exclusions — escreva exatamente como no campo "Driver" / nome do construtor
LOCK_DRIVERS = ["Oscar Piastri"]        # capitão 2×
EXCLUDE_DRIVERS = []              # ex.: ["Jack Doohan"]
LOCK_CONSTRUCTORS = ["McLaren", "Red Bull Racing"]
EXCLUDE_CONSTRUCTORS = []         # ex.: []

# Multiplicadores por piloto (nome -> fator). Ex.: {"Oscar Piastri": 2.0}
DRIVER_MULTIPLIERS = {
    # "Oscar Piastri": 2.0,
}


In [189]:

import os, glob
print("📂 Pasta:", RACE_TRACES_DIR)
files_top = os.listdir(RACE_TRACES_DIR)
print("Itens no nível atual:", len(files_top))
print("\nAlguns itens:")
for f in files_top[:20]:
    print(" -", f)

# Se os arquivos estiverem em subpastas (ex.: laps_2025), liste recursivamente:
all_files = glob.glob(RACE_TRACES_DIR + "/**/*.*", recursive=True)
print("\nTotal de arquivos (recursivo):", len(all_files))
print("Exemplos:")
for f in all_files[:10]:
    print(" -", f)

import os, glob, random
import pandas as pd
import numpy as np
from itertools import combinations

SEED = 42
random.seed(SEED); np.random.seed(SEED)

def safe_read(path):
    try:
        if path.lower().endswith('.csv'):
            return pd.read_csv(path)
        if path.lower().endswith('.parquet'):
            return pd.read_parquet(path)
        if path.lower().endswith('.json'):
            return pd.read_json(path, lines=False)
    except Exception as e:
        print(f"[warn] erro ao ler {path}: {e}")
    return None

def coalesce(series, default=np.nan):
    return series if series is not None else default


📂 Pasta: C:/F1/setup/f1_race_traces_2021/f1_race_traces_2021/data/laps_2025
Itens no nível atual: 21

Alguns itens:
 - laps_2025_Australian_Grand_Prix.csv
 - laps_2025_Austrian_Grand_Prix.csv
 - laps_2025_Azerbaijan_Grand_Prix.csv
 - laps_2025_Bahrain_Grand_Prix.csv
 - laps_2025_Belgian_Grand_Prix.csv
 - laps_2025_British_Grand_Prix.csv
 - laps_2025_Canadian_Grand_Prix.csv
 - laps_2025_Chinese_Grand_Prix.csv
 - laps_2025_Dutch_Grand_Prix.csv
 - laps_2025_Emilia_Romagna_Grand_Prix.csv
 - laps_2025_Hungarian_Grand_Prix.csv
 - laps_2025_Italian_Grand_Prix.csv
 - laps_2025_Japanese_Grand_Prix.csv
 - laps_2025_Mexico_City_Grand_Prix.csv
 - laps_2025_Miami_Grand_Prix.csv
 - laps_2025_Monaco_Grand_Prix.csv
 - laps_2025_Saudi_Arabian_Grand_Prix.csv
 - laps_2025_Singapore_Grand_Prix.csv
 - laps_2025_Spanish_Grand_Prix.csv
 - laps_2025_São_Paulo_Grand_Prix.csv

Total de arquivos (recursivo): 21
Exemplos:
 - C:/F1/setup/f1_race_traces_2021/f1_race_traces_2021/data/laps_2025\laps_2025_Australian_G

In [190]:

# =============
# 1) RACE TRACES
# =============
files = []
if os.path.isdir(RACE_TRACES_DIR):
    for ext in ('*.csv', '*.parquet', '*.json'):
        files.extend(glob.glob(os.path.join(RACE_TRACES_DIR, ext)))

dfs = []
for f in files:
    df = safe_read(f)
    if df is None or df.empty:
        continue

    # Inferência de corrida pelo nome do arquivo
    fname = os.path.basename(f)
    race_guess = None
    for name in ["Las Vegas","Jeddah","Miami","Singapore","Baku","Monaco","Abu Dhabi","Austin","Mexico City","Sao Paulo","Interlagos"]:
        if name.replace(" ", "_").lower() in fname.lower() or name.lower() in fname.lower():
            race_guess = name
            break

    # Mapear colunas possíveis
    driver = None
    for cand in ["driver","Driver","driver_name","abbr"]:
        if cand in df.columns:
            driver = df[cand].astype(str)
            break

    team = None
    for cand in ["team","constructor","Team","Constructor"]:
        if cand in df.columns:
            team = df[cand].astype(str)
            break

    start_pos = None
    for cand in ["grid","start_position","grid_position","startPos"]:
        if cand in df.columns:
            start_pos = pd.to_numeric(df[cand], errors='coerce')
            break

    finish_pos = None
    for cand in ["position","finish_position","classified_position","Pos"]:
        if cand in df.columns:
            finish_pos = pd.to_numeric(df[cand], errors='coerce')
            break

    fastest_lap_flag = None
    for cand in ["fastest_lap","is_fastest_lap","fastestLap"]:
        if cand in df.columns:
            fastest_lap_flag = (df[cand].astype(str).str.lower().isin(["1","true","yes"])).astype(int)
            break

    dnf_flag = None
    for cand in ["dnf","did_not_finish","retired","not_classified"]:
        if cand in df.columns:
            dnf_flag = (df[cand].astype(str).str.lower().isin(["1","true","yes"])).astype(int)
            break

    pit_time = None
    for cand in ["pit_time","pitstop_time","pitstopDuration","PitTime"]:
        if cand in df.columns:
            pit_time = pd.to_numeric(df[cand], errors='coerce')
            break

    if driver is None or (start_pos is None and finish_pos is None):
        continue

    out = pd.DataFrame({
        "driver": coalesce(driver),
        "team": coalesce(team),
        "race": race_guess,
        "start_position": coalesce(start_pos),
        "finish_position": coalesce(finish_pos),
        "fastest_lap": coalesce(fastest_lap_flag, 0),
        "dnf": coalesce(dnf_flag, 0),
        "pit_time": coalesce(pit_time, np.nan),
    })
    dfs.append(out)

if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
else:
    df_all = pd.DataFrame(columns=["driver","team","race","start_position","finish_position","fastest_lap","dnf","pit_time"])

for col in ["start_position","finish_position","fastest_lap","dnf","pit_time"]:
    if col in df_all.columns:
        df_all[col] = pd.to_numeric(df_all[col], errors='coerce')

# Agregação por piloto/corrida
if not df_all.empty:
    df_all["positions_gained"] = np.where(
        df_all["start_position"].notna() & df_all["finish_position"].notna(),
        (df_all["start_position"] - df_all["finish_position"]), np.nan
    )
    pit_summary = df_all.groupby(["driver","race"]).agg(
        pitstop_best=("pit_time","min"),
        pitstop_mean=("pit_time","mean")
    ).reset_index() if "pit_time" in df_all.columns else pd.DataFrame()

    base = df_all.groupby(["driver","race"]).agg(
        finish_position=("finish_position","min"),
        start_position=("start_position","min"),
        positions_gained=("positions_gained","mean"),
        fastest_lap=("fastest_lap","max"),
        dnf=("dnf","max"),
        laps_count=("finish_position","count")
    ).reset_index()

    if not pit_summary.empty:
        df_summary = base.merge(pit_summary, on=["driver","race"], how="left")
    else:
        df_summary = base.copy()
else:
    df_summary = pd.DataFrame(columns=["driver","race","finish_position","start_position","positions_gained","fastest_lap","dnf","laps_count","pitstop_best","pitstop_mean"])

df_summary.head(5)

,driver,race,finish_position,start_position,positions_gained,fastest_lap,dnf,laps_count,pitstop_best,pitstop_mean


In [191]:
# ==== VEPI ROBUSTO com fallback (usa df_summary se existir; senão usa 'merged') ====
import numpy as np
import pandas as pd

def _vepi_from_summary(df_summary):
    # Detectar coluna de nome da corrida
    NIGHT_TRACK_ALIASES = {
        "Las Vegas": ["Vegas"],
        "Miami": ["Miami", "Miami Gardens"],
        "Singapore": ["Singapore", "Marina Bay"],
        "Saudi": ["Jeddah", "Saudi"],
        "Abu Dhabi": ["Yas Island", "Abu Dhabi"],
        "Bahrain": ["Sakhir", "Bahrain"],
        "Qatar": ["Lusail", "Qatar"],
    }
    def is_night_urban(name: str) -> bool:
        if not isinstance(name, str): return False
        low = name.casefold()
        for aliases in NIGHT_TRACK_ALIASES.values():
            for a in aliases:
                if a.casefold() in low:
                    return True
        return False

    POSSIBLE_NAME_COLS = ["race", "grand_prix", "event", "session_name", "file"]
    NAME_COL = next((c for c in POSSIBLE_NAME_COLS if c in df_summary.columns), None)
    if NAME_COL is None:
        NAME_COL = "race"
        df_summary[NAME_COL] = ""

    mask_night = df_summary[NAME_COL].astype(str).apply(is_night_urban)
    df_urban = df_summary[mask_night].copy()
    if df_urban.empty:
        print("[aviso] Sem corridas noturnas detectadas; usando fallback com TODAS as corridas.")
        df_urban = df_summary.copy()

    vepi = df_urban.groupby("driver").agg(
        mean_positions_gained=("positions_gained","mean"),
        mean_finish_position=("finish_position","mean"),
        fastest_lap_count=("fastest_lap","sum"),
        dnf_rate=("dnf","mean"),
        pit_best_mean=("pitstop_best","mean"),
    )
    vepi = vepi.fillna({"mean_positions_gained":0.0, "dnf_rate":0.0})
    vepi["score_finish"]  = (25 - vepi["mean_finish_position"].fillna(20)).clip(0, 25)
    vepi["score_gain"]    = vepi["mean_positions_gained"] * 0.8
    vepi["score_fastlap"] = (vepi["fastest_lap_count"].fillna(0).clip(upper=1) > 0).astype(int) * 5
    vepi["score_dnf"]     = - vepi["dnf_rate"] * 20

    bins = [-np.inf, 2.00, 2.20, 2.50, 2.99, np.inf]
    vals = [ +4, +2, +1, 0, -1 ]
    vepi["score_pit"] = pd.cut(vepi["pit_best_mean"].fillna(3.2), bins=bins, labels=vals).astype(float)

    vepi["VEPI"] = vepi[["score_finish","score_gain","score_fastlap","score_dnf","score_pit"]].sum(axis=1)
    return vepi

# ===== Tenta usar df_summary; se não der, cai para 'merged' =====
if 'df_summary' in globals() and isinstance(df_summary, pd.DataFrame) and not df_summary.empty:
    vepi = _vepi_from_summary(df_summary)
    print("✅ VEPI calculado a partir de df_summary.")
else:
    # Fallback: deriva VEPI de 'merged' (proxy com base nas projeções do modelo)
    assert 'merged' in globals() and not merged.empty, "Nem df_summary nem merged estão prontos."
    pts_col = None
    for c in ["Projected Vegas Pts (no mult)", "Projected Vegas Pts", "Proj Pts (Event)"]:
        if c in merged.columns:
            pts_col = c
            break
    if pts_col is None:
        raise RuntimeError("Não encontrei coluna de pontos projetados em 'merged'.")

    tmp = merged.groupby("Driver", as_index=False).agg(
        proj_mean=(pts_col, "mean"),
        proj_std=(pts_col, "std"),
        cost=("Cost ($M)", "mean")
    )
    tmp["proj_std"] = tmp["proj_std"].fillna(0.0)
    # Proxy de VEPI: prioriza pontos esperados e leve penalidade por custo/risco
    tmp["VEPI"] = tmp["proj_mean"] - 0.05*tmp["cost"] - 0.25*tmp["proj_std"]
    vepi = tmp.set_index("Driver")[["VEPI"]]
    vepi.index.name = "driver"
    print("⚠️ VEPI em modo fallback (baseado em 'merged').")

display(vepi.sort_values("VEPI", ascending=False).head(10))


⚠️ VEPI em modo fallback (baseado em 'merged').


,VEPI
driver,
Lando Norris,29.180
Oscar Piastri,27.950
Max Verstappen,26.340
George Russell,22.680
Lewis Hamilton,16.160
Charles Leclerc,16.135
Kimi Antonelli,9.735
Oliver Bearman,8.435
Alexander Albon,8.390


In [192]:

# =============
# 2) VEPI (versão com aliases de pistas noturnas/urbanas)
# =============
import numpy as np

# Lista de aliases para identificar pistas noturnas reais
NIGHT_TRACK_ALIASES = {
    "Las Vegas": ["Vegas"],
    "Miami": ["Miami", "Miami Gardens"],
    "Singapore": ["Singapore", "Marina Bay"],
    "Saudi": ["Jeddah", "Saudi"],
    "Abu Dhabi": ["Yas Island", "Abu Dhabi"],
    "Bahrain": ["Sakhir", "Bahrain"],
    "Qatar": ["Lusail", "Qatar"],
}

def is_night_urban(name: str) -> bool:
    if not isinstance(name, str):
        return False
    low = name.casefold()
    for aliases in NIGHT_TRACK_ALIASES.values():
        for a in aliases:
            if a.casefold() in low:
                return True
    return False

if not df_summary.empty:
    # Detectar a coluna de nome de corrida
    POSSIBLE_NAME_COLS = ["race", "grand_prix", "event", "session_name", "file"]
    NAME_COL = next((c for c in POSSIBLE_NAME_COLS if c in df_summary.columns), None)
    if NAME_COL is None:
        NAME_COL = "race"
        df_summary[NAME_COL] = ""

    # Filtrar corridas noturnas/urbanas
    mask_night = df_summary[NAME_COL].apply(is_night_urban)
    df_urban = df_summary[mask_night].copy()

    if df_urban.empty:
        print("[aviso] Sem corridas noturnas detectadas, usando fallback com todas as corridas.")
        df_urban = df_summary.copy()

    vepi = df_urban.groupby("driver").agg(
        mean_positions_gained=("positions_gained", "mean"),
        mean_finish_position=("finish_position", "mean"),
        fastest_lap_count=("fastest_lap", "sum"),
        dnf_rate=("dnf", "mean"),
        pit_best_mean=("pitstop_best", "mean"),
    )

    vepi = vepi.fillna({"mean_positions_gained": 0.0, "dnf_rate": 0.0})

    vepi["score_finish"] = (25 - vepi["mean_finish_position"].fillna(20)).clip(lower=0, upper=25)
    vepi["score_gain"] = vepi["mean_positions_gained"] * 0.8
    vepi["score_fastlap"] = (vepi["fastest_lap_count"].fillna(0).clip(upper=1) > 0).astype(int) * 5
    vepi["score_dnf"] = - vepi["dnf_rate"] * 20

    bins = [-np.inf, 2.00, 2.20, 2.50, 2.99, np.inf]
    vals = [ +4, +2, +1, 0, -1 ]
    vepi["score_pit"] = pd.cut(vepi["pit_best_mean"].fillna(3.2), bins=bins, labels=vals).astype(float)

    vepi["VEPI"] = vepi[["score_finish","score_gain","score_fastlap","score_dnf","score_pit"]].sum(axis=1)

else:
    vepi = pd.DataFrame(columns=["driver","VEPI"])

# Visual rápido
vepi.sort_values("VEPI", ascending=False).head(10)



,driver,VEPI


In [193]:

# =============
# 3) Inputs Fantasy (fallback se não houver Excel)
# =============
fallback_data = [
    ("Lando Norris", 30.4, 30.70),
    ("Oscar Piastri", 26.0, 29.25),
    ("Max Verstappen", 29.2, 27.80),
    ("George Russell", 23.4, 23.85),
    ("Lewis Hamilton", 22.8, 17.30),
    ("Charles Leclerc", 23.3, 17.30),
    ("Kimi Antonelli", 16.3, 10.55),
    ("Alexander Albon", 13.2, 9.05),
    ("Oliver Bearman", 8.3, 8.85),
    ("Yuki Tsunoda", 10.8, 8.00),
    ("Lance Stroll", 8.5, 7.25),
    ("Nico Hülkenberg", 6.8, 6.25),
    ("Esteban Ocon", 6.5, 6.20),
    ("Isack Hadjar", 5.9, 4.15),
    ("Gabriel Bortoleto", 7.3, 3.85),
    ("Carlos Sainz", 6.3, 3.45),
    ("Liam Lawson", 16.8, 2.50),
    ("Pierre Gasly", 4.7, 1.65),
    ("Franco Colapinto", 4.7, 1.29),
    ("Fernando Alonso", 5.5, 1.25),
    ("Jack Doohan", 4.5, 0.00),
]
columns = ["Driver","Cost ($M)","Avg Pts/GP"]

if os.path.exists(FANTASY_INPUT_XLSX):
    try:
        df_inputs = pd.read_excel(FANTASY_INPUT_XLSX, sheet_name="1-Inputs (Drivers)")
    except Exception as e:
        print(f"[warn] Erro lendo {FANTASY_INPUT_XLSX}: {e}")
        df_inputs = pd.DataFrame(fallback_data, columns=columns)
else:
    df_inputs = pd.DataFrame(fallback_data, columns=columns)

if SPRINT_WEEKEND:
    df_inputs["Proj Pts (Event)"] = (df_inputs["Avg Pts/GP"] * SPRINT_UPLIFT).round(2)
else:
    df_inputs["Proj Pts (Event)"] = df_inputs["Avg Pts/GP"].round(2)

# Merge VEPI (ajuste percentual por desvio-padrão)
if not vepi.empty:
    vepi_norm = vepi[["VEPI"]].copy()
    vepi_norm["VEPI_z"] = (vepi_norm["VEPI"] - vepi_norm["VEPI"].mean()) / (vepi_norm["VEPI"].std(ddof=0) + 1e-6)
    vepi_norm["VEPI_adj"] = 1 + (vepi_norm["VEPI_z"] * 0.10)
    merged = df_inputs.merge(vepi_norm[["VEPI_adj"]], left_on="Driver", right_index=True, how="left")
    merged["VEPI_adj"].fillna(1.0, inplace=True)
else:
    merged = df_inputs.copy()
    merged["VEPI_adj"] = 1.0

merged["Projected Vegas Pts (no mult)"] = (merged["Proj Pts (Event)"] * merged["VEPI_adj"]).round(2)
merged.head(8)

,Driver,Cost ($M),Avg Pts/GP,Pts per $M,Proj Pts (Sprint wknd),Value (Sprint pts per $M),Proj Pts (Event),VEPI_adj,Projected Vegas Pts (no mult)
0,Lando Norris,30.4,30.70,1.01,38.38,1.263,30.70,1.0,30.70
1,Oscar Piastri,26.0,29.25,1.13,36.56,1.406,29.25,1.0,29.25
2,Max Verstappen,29.2,27.80,0.95,34.75,1.190,27.80,1.0,27.80
3,George Russell,23.4,23.85,1.02,29.81,1.274,23.85,1.0,23.85
4,Lewis Hamilton,22.8,17.30,0.76,21.62,0.948,17.30,1.0,17.30
5,Charles Leclerc,23.3,17.30,0.74,21.62,0.928,17.30,1.0,17.30
6,Kimi Antonelli,16.3,10.55,0.65,13.19,0.809,10.55,1.0,10.55
7,Alexander Albon,13.2,9.05,0.69,11.31,0.857,9.05,1.0,9.05


In [194]:

# =============
# 4) Mapear times e configurar construtores
# =============
# Mapeamento leve 2025 (ajuste se necessário)
TEAM_BY_DRIVER = {
    "Lando Norris": "McLaren",
    "Oscar Piastri": "McLaren",
    "Max Verstappen": "Red Bull Racing",
    "Liam Lawson": "Red Bull Racing",
    "George Russell": "Mercedes",
    "Kimi Antonelli": "Mercedes",
    "Lewis Hamilton": "Ferrari",
    "Charles Leclerc": "Ferrari",
    "Alexander Albon": "Williams",
    "Franco Colapinto": "Williams",
    "Nico Hülkenberg": "Kick Sauber",
    "Oliver Bearman": "Haas",  # ajustar se necessário
    "Esteban Ocon": "Alpine",
    "Pierre Gasly": "Alpine",
    "Isack Hadjar": "RB",
    "Yuki Tsunoda": "RB",
    "Lance Stroll": "Aston Martin",
    "Fernando Alonso": "Aston Martin",
    "Gabriel Bortoleto": "???",
    "Carlos Sainz": "???",
    "Jack Doohan": "Alpine",
}

merged["Team"] = merged["Driver"].map(TEAM_BY_DRIVER).fillna("Unknown")

# Parâmetros de pit dos construtores (faixas e bônus). Ajuste conforme seu modelo.
CONSTRUCTOR_COSTS = {
    "McLaren": 35.6,
    "Red Bull Racing": 30.2,
    "Ferrari": 33.0,
    "Mercedes": 32.0,
    "Williams": 18.0,
    "Aston Martin": 20.0,
    "Alpine": 15.0,
    "RB": 14.0,
    "Kick Sauber": 12.0,
    "Haas": 11.0,
    "Unknown": 10.0,
}

# Estimativas de pit para Vegas (pode substituir por valores reais quando tiver)
# Base + bônus (fastest 5, world record 15)
CONSTRUCTOR_PIT_POINTS = {
    "McLaren": {"base": 10, "fastest": 5, "world_record": 0},
    "Red Bull Racing": {"base": 5, "fastest": 0, "world_record": 0},
    "Ferrari": {"base": 5, "fastest": 0, "world_record": 0},
    "Mercedes": {"base": 2, "fastest": 0, "world_record": 0},
    "Williams": {"base": 0, "fastest": 0, "world_record": 0},
    "Aston Martin": {"base": 2, "fastest": 0, "world_record": 0},
    "Alpine": {"base": 0, "fastest": 0, "world_record": 0},
    "RB": {"base": 2, "fastest": 0, "world_record": 0},
    "Kick Sauber": {"base": 0, "fastest": 0, "world_record": 0},
    "Haas": {"base": 0, "fastest": 0, "world_record": 0},
    "Unknown": {"base": 0, "fastest": 0, "world_record": 0},
}

# Pontos estimados do construtor = soma(pts dos dois pilotos da equipe) + pit points
constructor_scores = []
for team, cost in CONSTRUCTOR_COSTS.items():
    team_drivers = merged[merged["Team"] == team]
    # Soma dos 2 melhores (ou de todos da equipe se 2 exatamente)
    top2 = team_drivers.nlargest(2, "Projected Vegas Pts (no mult)")
    drivers_sum = top2["Projected Vegas Pts (no mult)"].sum()
    pit = CONSTRUCTOR_PIT_POINTS.get(team, {"base":0,"fastest":0,"world_record":0})
    pit_points = pit["base"] + pit["fastest"] + pit["world_record"]
    constructor_scores.append((team, cost, drivers_sum, pit_points, drivers_sum + pit_points))

df_constructors = pd.DataFrame(constructor_scores, columns=["Constructor","Cost ($M)","Sum Drivers Pts","Pit Pts","Constructor Pts (Est.)"])

# Aplicar locks/exclusions para construtores
if LOCK_CONSTRUCTORS:
    df_constructors = df_constructors[df_constructors["Constructor"].isin(LOCK_CONSTRUCTORS + [c for c in df_constructors["Constructor"] if c not in EXCLUDE_CONSTRUCTORS])]
if EXCLUDE_CONSTRUCTORS:
    df_constructors = df_constructors[~df_constructors["Constructor"].isin(EXCLUDE_CONSTRUCTORS)]

df_constructors.sort_values("Constructor Pts (Est.)", ascending=False).head(5)

,Constructor,Cost ($M),Sum Drivers Pts,Pit Pts,Constructor Pts (Est.)
0,McLaren,35.6,59.95,15,74.95
2,Ferrari,33.0,34.60,5,39.60
3,Mercedes,32.0,34.40,2,36.40
1,Red Bull Racing,30.2,30.30,5,35.30
7,RB,14.0,12.15,2,14.15


In [195]:

# =============
# 5) Otimizador com 5 pilotos + N construtores
# =============
cand = merged.dropna(subset=["Projected Vegas Pts (no mult)","Cost ($M)"]).copy()
cand = cand[cand["Cost ($M)"] > 0]

# Aplicar locks/exclusions de pilotos
if LOCK_DRIVERS:
    cand = cand[cand["Driver"].isin(LOCK_DRIVERS + [d for d in cand["Driver"] if d not in EXCLUDE_DRIVERS])]
if EXCLUDE_DRIVERS:
    cand = cand[~cand["Driver"].isin(EXCLUDE_DRIVERS)]

# Multiplicadores
def apply_mult(driver, pts):
    mult = DRIVER_MULTIPLIERS.get(driver, 1.0)
    return pts * mult

best = {"pts": -1, "drivers": None, "constructors": None, "cost": None}

driver_rows = list(cand.itertuples(index=False))
constructors_rows = list(df_constructors.itertuples(index=False))

def constructors_combos(rows, k):
    if k == 1:
        for r in rows:
            yield (r,)
    else:
        from itertools import combinations
        for combo in combinations(rows, k):
            yield combo

for drv_combo in combinations(driver_rows, 5):
    drivers_cost = sum(getattr(d, "_2") for d in drv_combo)  # Cost ($M)
    # early prune by budget (min constructor cost)
    min_const_cost = min([getattr(c, "_2") for c in constructors_rows]) * NUM_CONSTRUCTORS if constructors_rows else 0
    if drivers_cost + min_const_cost > TOTAL_BUDGET:
        continue

    drivers_pts = sum(apply_mult(d[0], getattr(d, "_6")) for d in drv_combo)  # Projected Vegas Pts (no mult) * mult

    for cons_combo in constructors_combos(constructors_rows, NUM_CONSTRUCTORS):
        cons_cost = sum(getattr(c, "_2") for c in cons_combo)
        total_cost = drivers_cost + cons_cost
        if total_cost > TOTAL_BUDGET:
            continue
        cons_pts = sum(getattr(c, "_5") for c in cons_combo)  # Constructor Pts (Est.)
        total_pts = drivers_pts + cons_pts
        if total_pts > best["pts"]:
            best = {"pts": total_pts, "drivers": drv_combo, "constructors": cons_combo, "cost": total_cost}

best_summary = None
if best["drivers"]:
    best_drivers_df = pd.DataFrame([{
        "Driver": d[0],
        "Team": d[8],
        "Cost ($M)": d[1],
        "Projected Vegas Pts (no mult)": d[6],
        "Multiplier": DRIVER_MULTIPLIERS.get(d[0], 1.0),
        "Projected Vegas Pts (with mult)": apply_mult(d[0], d[6]),
        "Value (pts/$M)": round(apply_mult(d[0], d[6]) / d[1], 3)
    } for d in best["drivers"]]).sort_values("Projected Vegas Pts (with mult)", ascending=False).reset_index(drop=True)

    best_constructors_df = pd.DataFrame([{
        "Constructor": c[0],
        "Cost ($M)": c[1],
        "Sum Drivers Pts": c[2],
        "Pit Pts": c[3],
        "Constructor Pts (Est.)": c[4]
    } for c in best["constructors"]]).sort_values("Constructor Pts (Est.)", ascending=False).reset_index(drop=True)

    best_summary = {
        "Total Cost ($M)": round(best["cost"], 2),
        "Total Projected Pts": round(best["pts"], 2),
        "Drivers Cost": round(best_drivers_df["Cost ($M)"].sum(), 2),
        "Constructors Cost": round(best_constructors_df["Cost ($M)"].sum(), 2),
        "Drivers Pts (with mult)": round(best_drivers_df["Projected Vegas Pts (with mult)"].sum(), 2),
        "Constructors Pts": round(best_constructors_df["Constructor Pts (Est.)"].sum(), 2),
    }

best_summary, best_drivers_df.head(10) if best_summary else None, best_constructors_df if best_summary else None

AttributeError: 'Pandas' object has no attribute '_5'

In [ ]:

# =============
# 6) Exportar Excel v2 (estilizado PT-BR + Equipe Ideal com totalizador)
# =============
import pandas as pd
import xlsxwriter

# ---- Preparos: renomear para PT-BR (Drivers Projections) ----
drivers_pt = merged.sort_values("Projected Vegas Pts (no mult)", ascending=False).rename(columns={
    "Driver":"Piloto",
    "Team":"Equipe",
    "Cost ($M)":"Custo ($M)",
    "Avg Pts/GP":"Pontos médios/GP",
    "Pts per $M":"Pontos por $M",
    "Proj Pts (Sprint wknd)":"Proj. (fim de semana Sprint)",
    "Value (Sprint pts per $M)":"Valor (Sprint pts/$M)",
    "Proj Pts (Event)":"Proj. (Evento)",
    "VEPI_adj":"VEPI (ajustado)",
    "Projected Vegas Pts (no mult)":"Proj. Vegas (sem multiplicador)",
    "ΔPrice_pred":"ΔPreço (previsto)",
    "NewPrice_est":"Preço pós-GP (prev.)"
})

# Garante ordem amigável (colunas faltantes são ignoradas)
cols_order = [
    "Piloto","Custo ($M)","Pontos médios/GP","Pontos por $M",
    "Proj. (fim de semana Sprint)","Valor (Sprint pts/$M)",
    "Proj. (Evento)","VEPI (ajustado)","Proj. Vegas (sem multiplicador)",
    "Equipe","ΔPreço (previsto)","Preço pós-GP (prev.)"
]
drivers_pt = drivers_pt[[c for c in cols_order if c in drivers_pt.columns]]

# Construtores PT
constructors_pt = df_constructors.sort_values("Constructor Pts (Est.)", ascending=False).rename(columns={
    "Constructor":"Construtor",
    "Cost ($M)":"Custo ($M)",
    "Constructor Pts (Est.)":"Pontos do Construtor",
    "ΔPrice_pred":"ΔPreço (previsto)",
    "NewPrice_est":"Preço pós-GP (prev.)"
})

# Best Drivers / Best Constructors PT
best_drivers_pt = best_drivers_df.rename(columns={
    "Driver":"Piloto", "Team":"Equipe", "Cost ($M)":"Custo ($M)",
    "Projected Pts (no mult)":"Proj. (sem mult)",
    "Multiplier":"Multiplicador",
    "Projected Pts (with mult)":"Proj. (com mult)",
    "ΔPrice_pred":"ΔPreço (previsto)",
    "NewPrice_est":"Preço pós-GP (prev.)",
    "Value (pts/$M)":"Valor (pts/$M)"
})
best_constructors_pt = best_constructors_df.rename(columns={
    "Constructor":"Construtor", "Cost ($M)":"Custo ($M)",
    "Constructor Pts (Est.)":"Pontos do Construtor",
    "ΔPrice_pred":"ΔPreço (previsto)",
    "NewPrice_est":"Preço pós-GP (prev.)"
})

with pd.ExcelWriter(OUT_EXCEL, engine="xlsxwriter") as writer:
    wb = writer.book
    # Estilos
    h1 = wb.add_format({"bold":True,"font_color":"white","bg_color":"#0F6CBD","align":"center","valign":"vcenter","border":1})
    h2 = wb.add_format({"bold":True,"font_color":"white","bg_color":"#1F2937","align":"center","valign":"vcenter","border":1})
    fm_money = wb.add_format({"num_format":"#,##0.00","border":1})
    fm_num2  = wb.add_format({"num_format":"#,##0.00","border":1})
    fm_num3  = wb.add_format({"num_format":"#,##0.000","border":1})
    fm_txt   = wb.add_format({"border":1})
    fm_total = wb.add_format({"bold":True,"bg_color":"#E5F0FF","border":1,"num_format":"#,##0.00"})

    # ---- Aba Drivers Projections (PT) ----
    drivers_pt.to_excel(writer, sheet_name="Drivers Projections", index=False)
    ws = writer.sheets["Drivers Projections"]
    for c, name in enumerate(drivers_pt.columns):  # cabeçalho
        ws.write(0, c, name, h1)
    # Largura/formatos básicos por nome
    def setfmt(sheet, df, colname, width, fmt):
        if colname in df.columns:
            j = df.columns.get_loc(colname)
            sheet.set_column(j, j, width, fmt)
    setfmt(ws, drivers_pt, "Piloto", 22, fm_txt)
    setfmt(ws, drivers_pt, "Equipe", 18, fm_txt)
    setfmt(ws, drivers_pt, "Custo ($M)", 12, fm_money)
    setfmt(ws, drivers_pt, "Pontos médios/GP", 16, fm_num2)
    setfmt(ws, drivers_pt, "Pontos por $M", 14, fm_num3)
    setfmt(ws, drivers_pt, "Proj. (fim de semana Sprint)", 22, fm_num2)
    setfmt(ws, drivers_pt, "Valor (Sprint pts/$M)", 20, fm_num3)
    setfmt(ws, drivers_pt, "Proj. (Evento)", 14, fm_num2)
    setfmt(ws, drivers_pt, "VEPI (ajustado)", 14, fm_num2)
    setfmt(ws, drivers_pt, "Proj. Vegas (sem multiplicador)", 24, fm_num2)
    setfmt(ws, drivers_pt, "ΔPreço (previsto)", 16, fm_num3)
    setfmt(ws, drivers_pt, "Preço pós-GP (prev.)", 18, fm_money)

    # ---- Aba VEPI (se existir) ----
    if 'vepi' in globals() and not vepi.empty:
        vepi.reset_index().rename(columns={"driver":"Piloto","VEPI":"VEPI"}).to_excel(writer, sheet_name="VEPI", index=False)
        ws_v = writer.sheets["VEPI"]
        ws_v.write(0, 0, "Piloto", h1); ws_v.write(0, 1, "VEPI", h1)
        ws_v.set_column(0, 0, 22, fm_txt); ws_v.set_column(1, 1, 14, fm_num2)

    # ---- Aba Constructors (PT) ----
    constructors_pt.to_excel(writer, sheet_name="Constructors", index=False)
    ws_c = writer.sheets["Constructors"]
    for c, name in enumerate(constructors_pt.columns):
        ws_c.write(0, c, name, h1)
    setfmt(ws_c, constructors_pt, "Construtor", 22, fm_txt)
    setfmt(ws_c, constructors_pt, "Custo ($M)", 12, fm_money)
    setfmt(ws_c, constructors_pt, "Pontos do Construtor", 18, fm_num2)
    setfmt(ws_c, constructors_pt, "ΔPreço (previsto)", 16, fm_num3)
    setfmt(ws_c, constructors_pt, "Preço pós-GP (prev.)", 18, fm_money)

    # ---- Abas de “Best” (PT) ----
    best_drivers_pt.to_excel(writer, sheet_name="Best Drivers", index=False)
    ws_bd = writer.sheets["Best Drivers"]
    for c, name in enumerate(best_drivers_pt.columns):
        ws_bd.write(0, c, name, h1)
    for col in best_drivers_pt.columns:
        j = best_drivers_pt.columns.get_loc(col)
        if "Custo" in col or "Preço" in col: ws_bd.set_column(j, j, 12, fm_money)
        elif "Proj." in col or "Valor" in col: ws_bd.set_column(j, j, 16, fm_num2)
        elif "ΔPreço" in col: ws_bd.set_column(j, j, 14, fm_num3)
        else: ws_bd.set_column(j, j, 18, fm_txt)

    best_constructors_pt.to_excel(writer, sheet_name="Best Constructors", index=False)
    ws_bc = writer.sheets["Best Constructors"]
    for c, name in enumerate(best_constructors_pt.columns):
        ws_bc.write(0, c, name, h1)
    for col in best_constructors_pt.columns:
        j = best_constructors_pt.columns.get_loc(col)
        if "Custo" in col or "Preço" in col: ws_bc.set_column(j, j, 12, fm_money)
        elif "Pontos" in col: ws_bc.set_column(j, j, 16, fm_num2)
        elif "ΔPreço" in col: ws_bc.set_column(j, j, 14, fm_num3)
        else: ws_bc.set_column(j, j, 18, fm_txt)

    # ---- Aba Best Summary (usa seu dict best_summary) ----
    if best_summary:
        best_summary_df = pd.DataFrame([best_summary])
        best_summary_df.to_excel(writer, sheet_name="Best Summary", index=False)
        ws_bs = writer.sheets["Best Summary"]
        for c, name in enumerate(best_summary_df.columns):
            ws_bs.write(0, c, name, h1)
        for col in best_summary_df.columns:
            j = best_summary_df.columns.get_loc(col)
            ws_bs.set_column(j, j, 20, fm_num2 if best_summary_df[col].dtype != 'O' else fm_txt)

    # ---- Aba Equipe Ideal (com totalizador) ----
    sheet_eq = "Equipe Ideal"
    eq_cols = ["Tipo","Nome","Equipe/Construtor","Custo ($M)","Pts (proj.)","ΔPreço (previsto)","Preço pós-GP (prev.)"]
    eq_rows = []
    # Pilotos
    for _, r in best_drivers_df.iterrows():
        eq_rows.append([
            "Piloto",
            r.get("Driver"),
            r.get("Team"),
            r.get("Cost ($M)"),
            r.get("Projected Pts (with mult)"),
            r.get("ΔPrice_pred"),
            r.get("NewPrice_est"),
        ])
    # Construtores
    for _, r in best_constructors_df.iterrows():
        eq_rows.append([
            "Construtor",
            r.get("Constructor"),
            r.get("Constructor"),
            r.get("Cost ($M)"),
            r.get("Constructor Pts (Est.)"),
            r.get("ΔPrice_pred"),
            r.get("NewPrice_est"),
        ])
    eq_df = pd.DataFrame(eq_rows, columns=eq_cols)
    eq_df.to_excel(writer, sheet_name=sheet_eq, index=False)
    ws_i = writer.sheets[sheet_eq]
    # Cabeçalho e colunas
    for c, name in enumerate(eq_cols):
        ws_i.write(0, c, name, h2)
    widths = [12, 22, 22, 12, 14, 16, 18]
    formats = [fm_txt, fm_txt, fm_txt, fm_money, fm_num2, fm_num3, fm_money]
    for c, w in enumerate(widths):
        ws_i.set_column(c, c, w, formats[c])
    # Totalizador (linha final)
    last_row = len(eq_df) + 1  # 1-based + header
    ws_i.write(last_row, 0, "Totais", fm_total)
    ws_i.write_formula(last_row, 3, f"=SUM(D2:D{last_row})", fm_total)  # Custo
    ws_i.write_formula(last_row, 4, f"=SUM(E2:E{last_row})", fm_total)  # Pontos
    ws_i.write_formula(last_row, 5, f"=SUM(F2:F{last_row})", fm_total)  # ΔPreço
    ws_i.write_formula(last_row, 6, f"=SUM(G2:G{last_row})", fm_total)  # Preço pós-GP

OUT_EXCEL


NameError: name 'best_summary' is not defined

In [ ]:
# === PATCH: Otimizador + Export garantidos ===
import pandas as pd, os, traceback
from itertools import combinations

# 0) Configurar saída ABSOLUTA (ajuste se quiser outro local)
OUT_EXCEL = r"C:/F1/setup/f1_race_traces_2021/F1_Fantasy_Vegas_Projections_v2.xlsx"

# 1) Escolher a coluna correta de pontos dos pilotos
PTS_COL = "Projected Vegas Pts (no mult)" if "Projected Vegas Pts (no mult)" in merged.columns else (
          "Projected Vegas Pts" if "Projected Vegas Pts" in merged.columns else None)
assert PTS_COL, "Não encontrei a coluna de pontos projetados em 'merged'."

# 2) Preparar candidatos de pilotos (com multiplicador)
cand = merged[["Driver","Team","Cost ($M)", PTS_COL]].dropna().copy()
cand.rename(columns={PTS_COL:"ProjPts"}, inplace=True)
cand["Mult"] = cand["Driver"].map(DRIVER_MULTIPLIERS).fillna(1.0)
cand["ProjPts_mult"] = cand["ProjPts"] * cand["Mult"]

# 3) Preparar construtores (se não existir, cria um mock básico)
if 'df_constructors' not in globals() or df_constructors.empty:
    # mock simples com base nas equipes presentes nos pilotos
    cons = cand.groupby("Team").agg(
        SumDriversPts=("ProjPts","sum")
    ).reset_index().rename(columns={"Team":"Constructor"})
    cons["Cost ($M)"] = 25.0   # custo genérico
    cons["Pit Pts"] = 0
    cons["Constructor Pts (Est.)"] = cons["SumDriversPts"]
    df_constructors = cons[["Constructor","Cost ($M)","SumDrivers Pts","Pit Pts","Constructor Pts (Est.)"]]

# 4) Otimização bruta: 5 pilotos + 1 construtor dentro do TOTAL_BUDGET
assert "TOTAL_BUDGET" in globals(), "Defina TOTAL_BUDGET na célula de parâmetros."
best = None

pilots = list(cand.itertuples(index=False))           # (Driver, Team, Cost, ProjPts, Mult, ProjPts_mult)
constructors = list(df_constructors.itertuples(index=False))  # (Constructor, Cost, SumDriversPts, PitPts, ConsPts)

for drv_combo in combinations(pilots, 5):
    cost_drv = sum(d._3 for d in drv_combo)
    for cons in constructors:
        total_cost = cost_drv + cons._2
        if total_cost <= TOTAL_BUDGET:
            pts_total = sum(d._6 for d in drv_combo) + cons._5
            if (best is None) or (pts_total > best["pts"]):
                best = {
                    "drivers": drv_combo,
                    "constructor": cons,
                    "cost": total_cost,
                    "pts": pts_total
                }

# 5) Mostrar resultado
if best is None:
    raise RuntimeError("Não foi possível montar time ≤ orçamento. Verifique TOTAL_BUDGET, custos e dados.")

best_drivers_df = pd.DataFrame([{
    "Driver": d._0, "Team": d._1, "Cost ($M)": d._3, 
    "Projected Pts (no mult)": d._4, "Multiplier": d._5, 
    "Projected Pts (with mult)": d._6, "Value (pts/$M)": round(d._6 / d._3, 3)
} for d in best["drivers"]]).sort_values("Projected Pts (with mult)", ascending=False).reset_index(drop=True)

best_constructor_df = pd.DataFrame([{
    "Constructor": best["constructor"]._0,
    "Cost ($M)": best["constructor"]._2,
    "Constructor Pts (Est.)": best["constructor"]._5
}])

summary = pd.DataFrame([{
    "Total Cost ($M)": round(best["cost"], 2),
    "Total Projected Pts": round(best["pts"], 2),
    "Drivers Cost": round(best_drivers_df["Cost ($M)"].sum(), 2),
    "Constructor Cost": float(best_constructor_df["Cost ($M)"].iloc[0]),
    "Drivers Pts (with mult)": round(best_drivers_df["Projected Pts (with mult)"].sum(), 2),
    "Constructor Pts": float(best_constructor_df["Constructor Pts (Est.)"].iloc[0]),
}])

print("=== EQUIPE ÓTIMA (≤ orçamento) ===")
print(best_drivers_df)
print(best_constructor_df)
print(summary)

# 6) Exportar Excel (com mensagens explícitas)
print("\n→ Gravando planilha final em:", OUT_EXCEL)
try:
    with pd.ExcelWriter(OUT_EXCEL, engine="xlsxwriter") as w:
        merged.sort_values(PTS_COL, ascending=False).to_excel(w, sheet_name="Drivers Projections", index=False)
        if 'vepi' in globals() and not vepi.empty:
            vepi.reset_index().to_excel(w, sheet_name="VEPI", index=False)
        if 'df_constructors' in globals() and not df_constructors.empty:
            df_constructors.sort_values("Constructor Pts (Est.)", ascending=False).to_excel(w, sheet_name="Constructors", index=False)
        best_drivers_df.to_excel(w, sheet_name="Best Drivers", index=False)
        best_constructor_df.to_excel(w, sheet_name="Best Constructor", index=False)
        summary.to_excel(w, sheet_name="Best Summary", index=False)
    print("✅ Sucesso! Arquivo salvo.")
except Exception:
    print("❌ Erro ao salvar:")
    traceback.print_exc()

import os; print("Existe?", os.path.exists(OUT_EXCEL))


AttributeError: 'Pandas' object has no attribute '_3'

In [ ]:
# === OTIMIZADOR FINAL – 5 pilotos + 2 construtores (cap $119.6M) ===
import pandas as pd, os, traceback
from itertools import combinations

# ----- Config (ajuste se quiser) -----
TOTAL_BUDGET = 119.6
NUM_CONSTRUCTORS = 2
LOCK_CONSTRUCTORS = ["McLaren", "Red Bull Racing"]   # use exatamente o nome que aparece em df_constructors["Constructor"]
LOCK_DRIVERS = ["Oscar Piastri"]                      # manter Piastri
DRIVER_MULTIPLIERS = {"Oscar Piastri": 2.0}          # capitão 2×

OUT_EXCEL = r"C:/F1/setup/f1_race_traces_2021/F1_Fantasy_Vegas_Projections_v2.xlsx"

# ----- Descobrir coluna de pontos projetados em 'merged' -----
PTS_COL = None
for c in ["Projected Vegas Pts (no mult)", "Projected Vegas Pts", "Proj Pts (Event)"]:
    if c in merged.columns:
        PTS_COL = c; break
if PTS_COL is None:
    raise RuntimeError("Não encontrei a coluna de pontos projetados em 'merged'.")

# ----- Preparar pilotos -----
cand = merged[["Driver","Team","Cost ($M)", PTS_COL]].dropna().copy()
cand.rename(columns={PTS_COL: "ProjPts"}, inplace=True)
cand["Multiplier"] = cand["Driver"].map(DRIVER_MULTIPLIERS).fillna(1.0)
cand["ProjPts_mult"] = cand["ProjPts"] * cand["Multiplier"]

# locks: marca e garante que eles não saem
cand["IsLock"] = cand["Driver"].isin(LOCK_DRIVERS)

# ----- Preparar construtores (normaliza nomes de colunas) -----
if 'df_constructors' not in globals() or df_constructors.empty:
    # fallback simples: cria construtores a partir dos pilotos
    cons = cand.groupby("Team", as_index=False).agg(
        SumDriversPts=("ProjPts", "sum")
    ).rename(columns={"Team":"Constructor"})
    cons["Cost ($M)"] = 25.0
    cons["ConstructorPts"] = cons["SumDriversPts"]
    dfc = cons[["Constructor","Cost ($M)","ConstructorPts"]].copy()
else:
    dfc = df_constructors.copy()
    rename_map = {"Constructor Pts (Est.)":"ConstructorPts", "Sum Drivers Pts":"SumDriversPts","Pit Pts":"PitPts"}
    dfc.rename(columns=rename_map, inplace=True)
    if "ConstructorPts" not in dfc.columns:
        # tenta derivar
        if "SumDriversPts" in dfc.columns and "PitPts" in dfc.columns:
            dfc["ConstructorPts"] = dfc["SumDriversPts"] + dfc["PitPts"]
        elif "SumDriversPts" in dfc.columns:
            dfc["ConstructorPts"] = dfc["SumDriversPts"]
        else:
            raise RuntimeError("df_constructors sem coluna de pontos estimados.")
    dfc = dfc[["Constructor","Cost ($M)","ConstructorPts"]].dropna()

# aplica locks de construtor
if LOCK_CONSTRUCTORS:
    dfc = dfc[dfc["Constructor"].isin(LOCK_CONSTRUCTORS)].copy()
if dfc.empty:
    raise RuntimeError("Nenhum construtor restante após locks/filtros. Verifique nomes em LOCK_CONSTRUCTORS.")

# ----- Otimização bruta (com locks) -----
best = None
pilot_list = cand.to_dict("records")
cons_list = dfc.to_dict("records")

# pilotos travados precisam estar no combo
locks = [p for p in pilot_list if p["IsLock"]]
if len(locks) > 5:
    raise RuntimeError("Mais de 5 pilotos travados.")
fixed_cost = sum(p["Cost ($M)"] for p in locks)
fixed_pts  = sum(p["ProjPts_mult"] for p in locks)

# pool dos demais
pool = [p for p in pilot_list if not p["IsLock"]]

for others in combinations(pool, 5 - len(locks)):
    drv_combo = locks + list(others)
    cost_drv = sum(d["Cost ($M)"] for d in drv_combo)
    pts_drv  = sum(d["ProjPts_mult"] for d in drv_combo)

    for cons_combo in combinations(cons_list, NUM_CONSTRUCTORS):
        cost_cons = sum(c["Cost ($M)"] for c in cons_combo)
        total_cost = cost_drv + cost_cons
        if total_cost <= TOTAL_BUDGET:
            pts_cons = sum(c["ConstructorPts"] for c in cons_combo)
            total_pts = pts_drv + pts_cons
            if (best is None) or (total_pts > best["pts"]):
                best = {"drivers": drv_combo, "constructors": cons_combo, "cost": total_cost, "pts": total_pts}

if best is None:
    raise RuntimeError("Não foi possível montar time ≤ orçamento. Ajuste cap/locks ou verifique custos.")

# ----- Saídas em DataFrame -----
best_drivers_df = pd.DataFrame([{
    "Driver": d["Driver"],
    "Team": d["Team"],
    "Cost ($M)": d["Cost ($M)"],
    "Projected Pts (no mult)": d["ProjPts"],
    "Multiplier": d["Multiplier"],
    "Projected Pts (with mult)": d["ProjPts_mult"],
    "Value (pts/$M)": round(d["ProjPts_mult"]/d["Cost ($M)"], 3)
} for d in best["drivers"]]).sort_values("Projected Pts (with mult)", ascending=False).reset_index(drop=True)

best_constructors_df = pd.DataFrame([{
    "Constructor": c["Constructor"],
    "Cost ($M)": c["Cost ($M)"],
    "Constructor Pts (Est.)": c["ConstructorPts"]
} for c in best["constructors"]]).sort_values("Constructor Pts (Est.)", ascending=False).reset_index(drop=True)

summary = pd.DataFrame([{
    "Total Cost ($M)": round(best["cost"], 2),
    "Total Projected Pts": round(best["pts"], 2),
    "Drivers Cost": round(best_drivers_df["Cost ($M)"].sum(), 2),
    "Constructors Cost": round(best_constructors_df["Cost ($M)"].sum(), 2),
    "Drivers Pts (with mult)": round(best_drivers_df["Projected Pts (with mult)"].sum(), 2),
    "Constructors Pts": round(best_constructors_df["Constructor Pts (Est.)"].sum(), 2),
}])

print("=== 🏎️ EQUIPE OTIMIZADA (≤ $119.6M, 2 construtores travados) ===")
display(best_drivers_df)
display(best_constructors_df)
display(summary)

# ----- Exportar Excel (com abas) -----
print("\n→ Gravando planilha final em:", OUT_EXCEL)
try:
    with pd.ExcelWriter(OUT_EXCEL, engine="xlsxwriter") as w:
        merged.sort_values(PTS_COL, ascending=False).to_excel(w, sheet_name="Drivers Projections", index=False)
        if 'vepi' in globals():
            vepi.reset_index().to_excel(w, sheet_name="VEPI", index=False)
        # construtores
        dfc.sort_values("ConstructorPts", ascending=False).to_excel(w, sheet_name="Constructors", index=False)
        best_drivers_df.to_excel(w, sheet_name="Best Drivers", index=False)
        best_constructors_df.to_excel(w, sheet_name="Best Constructors", index=False)
        summary.to_excel(w, sheet_name="Best Summary", index=False)
    print("✅ Sucesso! Arquivo salvo.")
    print("Existe?", os.path.exists(OUT_EXCEL))
except Exception:
    print("❌ Erro ao salvar:")
    traceback.print_exc()

=== 🏎️ EQUIPE OTIMIZADA (≤ $119.6M, 2 construtores travados) ===


,Driver,Team,Cost ($M),Projected Pts (no mult),Multiplier,Projected Pts (with mult),Value (pts/$M)
0,Oscar Piastri,McLaren,26.0,29.25,2.0,58.50,2.250
1,Oliver Bearman,Haas,8.3,8.85,1.0,8.85,1.066
2,Nico Hülkenberg,Kick Sauber,6.8,6.25,1.0,6.25,0.919
3,Esteban Ocon,Alpine,6.5,6.20,1.0,6.20,0.954
4,Isack Hadjar,RB,5.9,4.15,1.0,4.15,0.703


,Constructor,Cost ($M),Constructor Pts (Est.)
0,McLaren,35.6,74.95
1,Red Bull Racing,30.2,35.30


,Total Cost ($M),Total Projected Pts,Drivers Cost,Constructors Cost,Drivers Pts (with mult),Constructors Pts
0,119.3,194.2,53.5,65.8,83.95,110.25



→ Gravando planilha final em: C:/F1/setup/f1_race_traces_2021/F1_Fantasy_Vegas_Projections_v2.xlsx
✅ Sucesso! Arquivo salvo.
Existe? True


In [ ]:
# === F1 Fantasy: Price Momentum (ΔPreço previsto) – módulo plug-and-play ===
import pandas as pd
import numpy as np
import os, traceback

# ---------- Config ----------
# limite de variação por evento (ajuste conforme sua regra interna)
PRICE_CAP_EVENT = 0.60   # ±$0.60M por GP (ajuste se usa outro teto)
PRICE_SCALE     = 0.12   # sensibilidade base (quanto 1.0 de score vira Δ$)

OUT_EXCEL = r"C:/F1/setup/f1_race_traces_2021/F1_Fantasy_Vegas_Projections_v2.xlsx"

# Colunas possíveis de pontos projetados
PTS_CANDIDATES = ["Projected Vegas Pts (no mult)", "Projected Vegas Pts", "Proj Pts (Event)"]

# ---------- Helpers ----------
def zscore(s):
    s = pd.to_numeric(s, errors="coerce")
    m, v = s.mean(), s.std(ddof=0)
    if v == 0 or np.isnan(v): 
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - m) / v

def logistic(x):
    return 1.0 / (1.0 + np.exp(-x))

def pick_first(cols, options):
    for c in options:
        if c in cols: 
            return c
    return None

# ======================================================
# 1) Price Momentum para PILOTOS (DataFrame: merged)
# ======================================================
assert 'merged' in globals() and not merged.empty, "DataFrame 'merged' não encontrado."

pts_col = pick_first(merged.columns, PTS_CANDIDATES)
if not pts_col:
    raise RuntimeError("Não encontrei coluna de pontos projetados em 'merged'.")

drivers = merged.copy()

# Features básicas
drivers["ProjPts"]  = pd.to_numeric(drivers[pts_col], errors="coerce")
drivers["CostM"]    = pd.to_numeric(drivers.get("Cost ($M)"), errors="coerce")
drivers["VEPI"]     = pd.to_numeric(vepi["VEPI"]).reindex(drivers["Driver"]).values if 'vepi' in globals() and hasattr(vepi, 'reindex') else np.nan
drivers["SelPct"]   = pd.to_numeric(drivers.get("Selection %"), errors="coerce") if "Selection %" in drivers.columns else np.nan
drivers["DNF_rate"] = pd.to_numeric(drivers.get("DNF rate"), errors="coerce") if "DNF rate" in drivers.columns else np.nan

# Value per Million (se já existir, mantém)
if "Value (pts/$M)" in drivers.columns:
    drivers["ValuePerM"] = pd.to_numeric(drivers["Value (pts/$M)"], errors="coerce")
else:
    drivers["ValuePerM"] = drivers["ProjPts"] / drivers["CostM"]

# Z-scores (robustos a NaN)
Z_proj   = zscore(drivers["ProjPts"])
Z_vepi   = zscore(drivers["VEPI"])
Z_value  = zscore(drivers["ValuePerM"])
Z_sel    = zscore(drivers["SelPct"])
Z_dnf    = zscore(drivers["DNF_rate"])
Z_price  = zscore(drivers["CostM"])  # leve contrapeso para preços já “esticados”

# Score sintético de momentum (ajuste pesos conforme seu backtest)
score_drv = (
    0.50*Z_proj   +
    0.30*Z_vepi   +
    0.20*Z_value  +
    0.20*Z_sel    -
    0.30*Z_dnf    -
    0.15*Z_price
)

# ΔPreço previsto e probabilidade de alta
drivers["ΔPrice_pred"] = np.clip(score_drv * PRICE_SCALE, -PRICE_CAP_EVENT, PRICE_CAP_EVENT)
drivers["PriceUp_Prob"] = logistic(score_drv)  # 0–1
drivers["NewPrice_est"] = (drivers["CostM"] + drivers["ΔPrice_pred"]).round(2)

drivers_momentum = drivers[[
    "Driver","Team","CostM","ProjPts","VEPI","ValuePerM","SelPct","DNF_rate",
    "ΔPrice_pred","PriceUp_Prob","NewPrice_est"
]].rename(columns={
    "CostM":"Cost ($M)",
    "SelPct":"Selection %",
})

# ======================================================
# 2) Price Momentum para CONSTRUTORES (DataFrame: df_constructors)
# ======================================================
assert 'df_constructors' in globals() and not df_constructors.empty, "DataFrame 'df_constructors' não encontrado."

dfc = df_constructors.copy()
# Normaliza nomes se necessário
dfc = dfc.rename(columns={
    "Constructor Pts (Est.)":"ConstructorPts",
    "Sum Drivers Pts":"SumDriversPts",
    "Pit Pts":"PitPts",
    "Cost ($M)":"Cost ($M)"
})

# Features básicas
dfc["ConstructorPts"] = pd.to_numeric(dfc.get("ConstructorPts"), errors="coerce")
dfc["CostM"]          = pd.to_numeric(dfc.get("Cost ($M)"), errors="coerce")
dfc["PitPts"]         = pd.to_numeric(dfc.get("PitPts"), errors="coerce") if "PitPts" in dfc.columns else 0.0
dfc["SelPct"]         = pd.to_numeric(dfc.get("Selection %"), errors="coerce") if "Selection %" in dfc.columns else np.nan

# Z-scores
Z_cpts  = zscore(dfc["ConstructorPts"])
Z_cval  = zscore(dfc["ConstructorPts"]/dfc["CostM"])
Z_csel  = zscore(dfc["SelPct"])
Z_cprice= zscore(dfc["CostM"])

score_cons = (
    0.60*Z_cpts +
    0.25*Z_cval +
    0.20*Z_csel -
    0.15*Z_cprice
)

dfc["ΔPrice_pred"]   = np.clip(score_cons * PRICE_SCALE, -PRICE_CAP_EVENT, PRICE_CAP_EVENT)
dfc["PriceUp_Prob"]  = logistic(score_cons)
dfc["NewPrice_est"]  = (dfc["CostM"] + dfc["ΔPrice_pred"]).round(2)

constructors_momentum = dfc[[
    "Constructor","Cost ($M)","ConstructorPts","PitPts","Selection %","ΔPrice_pred","PriceUp_Prob","NewPrice_est"
]] if "Selection %" in dfc.columns else dfc[[
    "Constructor","Cost ($M)","ConstructorPts","PitPts","ΔPrice_pred","PriceUp_Prob","NewPrice_est"
]]

# ======================================================
# 3) Mostrar top risers / droppers e gravar no Excel
# ======================================================
top_up_drv   = drivers_momentum.sort_values("ΔPrice_pred", ascending=False).head(10)
top_down_drv = drivers_momentum.sort_values("ΔPrice_pred", ascending=True).head(10)
top_up_con   = constructors_momentum.sort_values("ΔPrice_pred", ascending=False).head(5)
top_down_con = constructors_momentum.sort_values("ΔPrice_pred", ascending=True).head(5)

print("🏎️ Top 10 Drivers – possíveis altas de preço:")
display(top_up_drv[["Driver","Team","Cost ($M)","ProjPts","VEPI","ΔPrice_pred","PriceUp_Prob","NewPrice_est"]])

print("🏎️ Top 10 Drivers – possíveis quedas de preço:")
display(top_down_drv[["Driver","Team","Cost ($M)","ProjPts","VEPI","ΔPrice_pred","PriceUp_Prob","NewPrice_est"]])

print("🏁 Top 5 Constructors – possíveis altas:")
display(top_up_con[["Constructor","Cost ($M)","ConstructorPts","ΔPrice_pred","PriceUp_Prob","NewPrice_est"]])

print("🏁 Top 5 Constructors – possíveis quedas:")
display(top_down_con[["Constructor","Cost ($M)","ConstructorPts","ΔPrice_pred","PriceUp_Prob","NewPrice_est"]])

# ---- Exportar/Anexar ao Excel final (mantém abas anteriores) ----
try:
    # Se o arquivo já existir, reescreve todas as abas relevantes (para não abrir handle em modo append)
    with pd.ExcelWriter(OUT_EXCEL, engine="xlsxwriter") as w:
        # 1) Pilotos (com momentum)
        merged_out = merged.copy()
        merged_out = merged_out.merge(
            drivers_momentum[["Driver","ΔPrice_pred","PriceUp_Prob","NewPrice_est"]],
            on="Driver", how="left"
        )
        merged_out.to_excel(w, sheet_name="Drivers Projections", index=False)

        # 2) VEPI (se existir)
        if 'vepi' in globals():
            vepi.reset_index().to_excel(w, sheet_name="VEPI", index=False)

        # 3) Construtores (com momentum)
        dfc_out = dfc.copy()
        dfc_out.to_excel(w, sheet_name="Constructors", index=False)

        # 4) Price Momentum – visão consolidada
        with pd.ExcelWriter(OUT_EXCEL, engine="xlsxwriter", mode="w") as _:
            pass  # garante lock release imediato
    # Segunda passada: escreve todas as abas incluindo os rankeamentos (evita planilha muito grande na primeira)
    with pd.ExcelWriter(OUT_EXCEL, engine="xlsxwriter") as w:
        merged_out.to_excel(w, sheet_name="Drivers Projections", index=False)
        if 'vepi' in globals():
            vepi.reset_index().to_excel(w, sheet_name="VEPI", index=False)
        dfc_out.to_excel(w, sheet_name="Constructors", index=False)

        # Nova aba consolidada de Momentum
        with pd.ExcelWriter(OUT_EXCEL, engine="xlsxwriter", mode="a") as _:
            pass
    # Como 'xlsxwriter' não suporta append nativo facilmente, criamos uma planilha separada:
    MOMENTUM_PATH = OUT_EXCEL.replace(".xlsx", "_withMomentum.xlsx")
    with pd.ExcelWriter(MOMENTUM_PATH, engine="xlsxwriter") as w:
        merged_out.to_excel(w, sheet_name="Drivers Projections", index=False)
        if 'vepi' in globals():
            vepi.reset_index().to_excel(w, sheet_name="VEPI", index=False)
        dfc_out.to_excel(w, sheet_name="Constructors", index=False)
        drivers_momentum.to_excel(w, sheet_name="Price Momentum (Drivers)", index=False)
        constructors_momentum.to_excel(w, sheet_name="Price Momentum (Cons)", index=False)
        top_up_drv.to_excel(w, sheet_name="Top Up Drivers", index=False)
        top_down_drv.to_excel(w, sheet_name="Top Down Drivers", index=False)
        top_up_con.to_excel(w, sheet_name="Top Up Cons", index=False)
        top_down_con.to_excel(w, sheet_name="Top Down Cons", index=False)

    print(f"✅ Momentum salvo em: {MOMENTUM_PATH}")
    print("ℹ️ Se o arquivo original estava aberto, use o *_withMomentum.xlsx*.")    
except PermissionError:
    ts = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
    MOMENTUM_PATH = OUT_EXCEL.replace(".xlsx", f"_withMomentum_{ts}.xlsx")
    with pd.ExcelWriter(MOMENTUM_PATH, engine="xlsxwriter") as w:
        merged_out.to_excel(w, sheet_name="Drivers Projections", index=False)
        if 'vepi' in globals():
            vepi.reset_index().to_excel(w, sheet_name="VEPI", index=False)
        dfc_out.to_excel(w, sheet_name="Constructors", index=False)
        drivers_momentum.to_excel(w, sheet_name="Price Momentum (Drivers)", index=False)
        constructors_momentum.to_excel(w, sheet_name="Price Momentum (Cons)", index=False)
        top_up_drv.to_excel(w, sheet_name="Top Up Drivers", index=False)
        top_down_drv.to_excel(w, sheet_name="Top Down Drivers", index=False)
        top_up_con.to_excel(w, sheet_name="Top Up Cons", index=False)
        top_down_con.to_excel(w, sheet_name="Top Down Cons", index=False)
    print(f"⚠️ Arquivo em uso; salvei como: {MOMENTUM_PATH}")
except Exception:
    print("❌ Erro ao salvar Momentum:")
    traceback.print_exc()


🏎️ Top 10 Drivers – possíveis altas de preço:


,Driver,Team,Cost ($M),ProjPts,VEPI,ΔPrice_pred,PriceUp_Prob,NewPrice_est
1,Oscar Piastri,McLaren,26.0,29.25,27.950,0.195548,0.836110,26.20
0,Lando Norris,McLaren,30.4,30.70,29.180,0.191517,0.831455,30.59
2,Max Verstappen,Red Bull Racing,29.2,27.80,26.340,0.160428,0.791979,29.36
3,George Russell,Mercedes,23.4,23.85,22.680,0.138629,0.760466,23.54
4,Lewis Hamilton,Ferrari,22.8,17.30,16.160,0.053739,0.610121,22.85
5,Charles Leclerc,Ferrari,23.3,17.30,16.135,0.051366,0.605408,23.35
8,Oliver Bearman,Haas,8.3,8.85,8.435,0.024317,0.550488,8.32
12,Esteban Ocon,Alpine,6.5,6.20,5.875,-0.007037,0.485344,6.49
6,Kimi Antonelli,Mercedes,16.3,10.55,9.735,-0.008475,0.482350,16.29
10,Lance Stroll,Aston Martin,8.5,7.25,6.825,-0.008605,0.482081,8.49


🏎️ Top 10 Drivers – possíveis quedas de preço:


,Driver,Team,Cost ($M),ProjPts,VEPI,ΔPrice_pred,PriceUp_Prob,NewPrice_est
20,Jack Doohan,Alpine,4.5,0.00,-0.225,-0.138013,0.240469,4.36
16,Liam Lawson,Red Bull Racing,16.8,2.50,1.660,-0.129189,0.254155,16.67
19,Fernando Alonso,Aston Martin,5.5,1.25,0.975,-0.110295,0.285136,5.39
18,Franco Colapinto,Williams,4.7,1.29,1.055,-0.104484,0.295109,4.60
17,Pierre Gasly,Alpine,4.7,1.65,1.415,-0.095000,0.311810,4.60
15,Carlos Sainz,???,6.3,3.45,3.135,-0.065404,0.367017,6.23
14,Gabriel Bortoleto,???,7.3,3.85,3.485,-0.065153,0.367503,7.23
13,Isack Hadjar,RB,5.9,4.15,3.855,-0.045538,0.406252,5.85
9,Yuki Tsunoda,RB,10.8,8.00,7.460,-0.014754,0.469301,10.79
7,Alexander Albon,Williams,13.2,9.05,8.390,-0.013742,0.471402,13.19


🏁 Top 5 Constructors – possíveis altas:


,Constructor,Cost ($M),ConstructorPts,ΔPrice_pred,PriceUp_Prob,NewPrice_est
0,McLaren,35.6,74.95,0.221859,0.863989,35.82
2,Ferrari,33.0,39.60,0.054882,0.612386,33.05
1,Red Bull Racing,30.2,35.30,0.043876,0.590404,30.24
3,Mercedes,32.0,36.40,0.042353,0.587331,32.04
7,RB,14.0,14.15,-0.005868,0.487777,13.99


🏁 Top 5 Constructors – possíveis quedas:


,Constructor,Cost ($M),ConstructorPts,ΔPrice_pred,PriceUp_Prob,NewPrice_est
10,Unknown,10.0,0.00,-0.104091,0.295791,9.90
5,Aston Martin,20.0,10.50,-0.057626,0.382201,19.94
6,Alpine,15.0,7.85,-0.057097,0.383243,14.94
8,Kick Sauber,12.0,6.25,-0.056899,0.383632,11.94
4,Williams,18.0,10.34,-0.051488,0.394350,17.95


❌ Erro ao salvar Momentum:


Traceback (most recent call last):
  File "C:\Users\wxamb\AppData\Local\Temp\ipykernel_31348\248460618.py", line 182, in <module>
    with pd.ExcelWriter(OUT_EXCEL, engine="xlsxwriter", mode="a") as _:
         ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\wxamb\AppData\Local\Programs\Python\Python314\Lib\site-packages\pandas\io\excel\_xlsxwriter.py", line 202, in __init__
    raise ValueError("Append mode is not supported with xlsxwriter!")
ValueError: Append mode is not supported with xlsxwriter!


In [ ]:
# --- Merge manual das colunas de valorização no merged e nos construtores ---
merged = merged.merge(
    drivers_momentum[["Driver","ΔPrice_pred","NewPrice_est"]],
    on="Driver", how="left"
)
df_constructors = df_constructors.merge(
    constructors_momentum[["Constructor","ΔPrice_pred","NewPrice_est"]],
    on="Constructor", how="left"
)
print("✅ Merge de ΔPrice_pred e NewPrice_est aplicado com sucesso!")
print(merged[["Driver","ΔPrice_pred","NewPrice_est"]].head())


✅ Merge de ΔPrice_pred e NewPrice_est aplicado com sucesso!
           Driver  ΔPrice_pred  NewPrice_est
0    Lando Norris     0.191517         30.59
1   Oscar Piastri     0.195548         26.20
2  Max Verstappen     0.160428         29.36
3  George Russell     0.138629         23.54
4  Lewis Hamilton     0.053739         22.85


In [ ]:
# ================================
# Otimizador v2 – Pontos + λ·Valorização + Transfers (EXPORT LIMPO)
# ================================
import pandas as pd, numpy as np, os, traceback, datetime
from itertools import combinations

# ---------- CONFIG ----------
TOTAL_BUDGET        = 119.6
NUM_CONSTRUCTORS    = 2
LOCK_CONSTRUCTORS   = ["McLaren", "Red Bull Racing"]   # use os nomes exatamente como estão no df_constructors
LOCK_DRIVERS        = ["Oscar Piastri"]
DRIVER_MULTIPLIERS  = {"Oscar Piastri": 2.0}           # capitão 2×

LAMBDA_GAIN         = 0.30      # peso da valorização (0.2–0.4)
FREE_TRANSFERS      = 2         # franquia
TRANSFER_PENALTY    = 10        # -10 por transfer extra
USE_WILDCARD        = False     # True => ignora penalidade nesta rodada
USE_FINAL_FIX       = False     # placeholder

# Informe seu elenco atual (opcional). Se vazio, não penaliza transfers.
CURRENT_DRIVERS       = []  # ex.: ["Oscar Piastri","Oliver Bearman","Esteban Ocon","Nico Hulkenberg","Pierre Gasly"]
CURRENT_CONSTRUCTORS  = []  # ex.: ["McLaren","Red Bull Racing"]

OUT_EXCEL = r"C:/F1/setup/f1_race_traces_2021/F1_Fantasy_Vegas_Projections_v2_withMomentum.xlsx"

# ---------- SANIDADE ----------
assert 'merged' in globals() and not merged.empty, "DataFrame 'merged' não encontrado."
assert 'df_constructors' in globals() and not df_constructors.empty, "DataFrame 'df_constructors' não encontrado."

PTS_COL = next((c for c in ["Projected Vegas Pts (no mult)", "Projected Vegas Pts", "Proj Pts (Event)"] if c in merged.columns), None)
if not PTS_COL:
    raise RuntimeError("Não encontrei coluna de pontos projetados em 'merged'.")

# ---------- DRIVERS ----------
drv = merged.copy()
drv["ProjPts"]        = pd.to_numeric(drv[PTS_COL], errors="coerce")
drv["CostM"]          = pd.to_numeric(drv.get("Cost ($M)"), errors="coerce")
drv["Multiplier"]     = drv["Driver"].map(DRIVER_MULTIPLIERS).fillna(1.0)
drv["ProjPts_mult"]   = drv["ProjPts"] * drv["Multiplier"]

# Valorização (módulo Momentum) – safe-get por coluna
if "ΔPrice_pred" in drv.columns:
    drv["ΔPrice_pred"] = pd.to_numeric(drv["ΔPrice_pred"], errors="coerce").fillna(0.0)
else:
    drv["ΔPrice_pred"] = 0.0

if "NewPrice_est" in drv.columns:
    drv["NewPrice_est"] = pd.to_numeric(drv["NewPrice_est"], errors="coerce")
else:
    drv["NewPrice_est"] = np.nan
drv["IsLock"]         = drv["Driver"].isin(LOCK_DRIVERS)

# ---------- CONSTRUTORES ----------
dfc = df_constructors.copy()
dfc.rename(columns={
    "Constructor Pts (Est.)": "ConstructorPts",
    "Sum Drivers Pts": "SumDriversPts",
    "Pit Pts": "PitPts",
}, inplace=True)
for col in ["Constructor","Cost ($M)","ConstructorPts"]:
    if col not in dfc.columns:
        if col == "ConstructorPts" and "SumDriversPts" in dfc.columns:
            dfc["ConstructorPts"] = pd.to_numeric(dfc["SumDriversPts"], errors="coerce").fillna(0.0) + pd.to_numeric(dfc.get("PitPts", 0.0), errors="coerce").fillna(0.0)
        else:
            dfc[col] = 0.0

if "ΔPrice_pred" in dfc.columns:
    dfc["ΔPrice_pred"] = pd.to_numeric(dfc["ΔPrice_pred"], errors="coerce").fillna(0.0)
else:
    dfc["ΔPrice_pred"] = 0.0

if "NewPrice_est" in dfc.columns:
    dfc["NewPrice_est"] = pd.to_numeric(dfc["NewPrice_est"], errors="coerce")
else:
    dfc["NewPrice_est"] = np.nan

# Aplicar locks de construtores
if LOCK_CONSTRUCTORS:
    dfc = dfc[dfc["Constructor"].isin(LOCK_CONSTRUCTORS)].copy()
if dfc.shape[0] != NUM_CONSTRUCTORS:
    raise RuntimeError(f"Esperava {NUM_CONSTRUCTORS} construtores travados; encontrei {dfc.shape[0]}. Verifique LOCK_CONSTRUCTORS conforme df_constructors['Constructor'].")

# ---------- AUXILIARES ----------
def count_transfers(selected_drivers, selected_constructors):
    if not CURRENT_DRIVERS and not CURRENT_CONSTRUCTORS:
        return 0
    cur_drv, cur_con = set(CURRENT_DRIVERS), set(CURRENT_CONSTRUCTORS)
    sel_drv, sel_con = set(selected_drivers), set(selected_constructors)
    drv_changes = len(cur_drv.symmetric_difference(sel_drv))
    con_changes = len(cur_con.symmetric_difference(sel_con))
    return int(np.ceil(drv_changes/2) + np.ceil(con_changes/2))

def objective(drv_combo, cons_combo):
    pts_drv  = sum(d["ProjPts_mult"] for d in drv_combo)
    pts_cons = sum(c["ConstructorPts"] for c in cons_combo)
    val_drv  = sum(d["ΔPrice_pred"] for d in drv_combo)
    val_cons = sum(c["ΔPrice_pred"] for c in cons_combo)
    obj = pts_drv + pts_cons + LAMBDA_GAIN*(val_drv + val_cons)
    if not USE_WILDCARD:
        trans = count_transfers([d["Driver"] for d in drv_combo], [c["Constructor"] for c in cons_combo])
        obj -= TRANSFER_PENALTY * max(0, trans - FREE_TRANSFERS)
    return obj

# ---------- OTIMIZAÇÃO ----------
candidates = drv[["Driver","Team","CostM","ProjPts","Multiplier","ProjPts_mult","ΔPrice_pred","NewPrice_est","IsLock"]] \
               .dropna(subset=["CostM","ProjPts"]).to_dict("records")
locks = [d for d in candidates if d["IsLock"]]
if len(locks) > 5:
    raise RuntimeError("Há mais de 5 pilotos travados.")
pool  = [d for d in candidates if not d["IsLock"]]

cons_list = dfc[["Constructor","Cost ($M)","ConstructorPts","ΔPrice_pred","NewPrice_est"]].to_dict("records")

min_cons_cost = sum(c["Cost ($M)"] for c in cons_list)
locks_cost    = sum(d["CostM"] for d in locks)
if locks_cost + min_cons_cost > TOTAL_BUDGET:
    raise RuntimeError(f"Locks + construtores excedem o cap (${locks_cost + min_cons_cost:.2f} > ${TOTAL_BUDGET:.2f}). Ajuste locks/cap.")

best = None
for others in combinations(pool, 5 - len(locks)):
    drv_combo = locks + list(others)
    cost_drv  = sum(d["CostM"] for d in drv_combo)
    if cost_drv + min_cons_cost > TOTAL_BUDGET:
        continue
    # Construtores travados -> único combo
    cons_combo = tuple(cons_list)
    total_cost = cost_drv + sum(c["Cost ($M)"] for c in cons_combo)
    if total_cost <= TOTAL_BUDGET:
        obj = objective(drv_combo, cons_combo)
        if (best is None) or (obj > best["obj"]):
            best = {"drivers": drv_combo, "constructors": cons_combo, "cost": total_cost, "obj": obj}

if best is None:
    raise RuntimeError("Não foi possível montar time ≤ orçamento. Revise cap/locks.")

# ---------- SAÍDAS ----------
best_drivers_df = pd.DataFrame([{
    "Driver": d["Driver"],
    "Team": d["Team"],
    "Cost ($M)": round(d["CostM"],2),
    "Projected Pts (no mult)": round(d["ProjPts"],2),
    "Multiplier": d["Multiplier"],
    "Projected Pts (with mult)": round(d["ProjPts_mult"],2),
    "ΔPrice_pred": round(d["ΔPrice_pred"],2),
    "NewPrice_est": round(d["NewPrice_est"],2) if pd.notna(d["NewPrice_est"]) else np.nan,
    "Value (pts/$M)": round(d["ProjPts_mult"]/d["CostM"],3) if d["CostM"] else np.nan
} for d in best["drivers"]]).sort_values("Projected Pts (with mult)", ascending=False).reset_index(drop=True)

best_constructors_df = pd.DataFrame([{
    "Constructor": c["Constructor"],
    "Cost ($M)": round(c["Cost ($M)"],2),
    "Constructor Pts (Est.)": round(c["ConstructorPts"],2),
    "ΔPrice_pred": round(c["ΔPrice_pred"],2),
    "NewPrice_est": round(c["NewPrice_est"],2) if pd.notna(c["NewPrice_est"]) else np.nan
} for c in best["constructors"]]).sort_values("Constructor Pts (Est.)", ascending=False).reset_index(drop=True)

total_pts_drv  = float(best_drivers_df["Projected Pts (with mult)"].sum())
total_pts_cons = float(best_constructors_df["Constructor Pts (Est.)"].sum())
total_gain     = float(best_drivers_df["ΔPrice_pred"].fillna(0).sum() + best_constructors_df["ΔPrice_pred"].fillna(0).sum())

transfers_used = 0 if (not CURRENT_DRIVERS and not CURRENT_CONSTRUCTORS) else count_transfers(
    best_drivers_df["Driver"].tolist(), best_constructors_df["Constructor"].tolist()
)
transfer_pen  = 0 if (USE_WILDCARD or transfers_used <= FREE_TRANSFERS) else TRANSFER_PENALTY * (transfers_used - FREE_TRANSFERS)

summary = pd.DataFrame([{
    "Total Cost ($M)": round(best["cost"], 2),
    "Total Projected Pts": round(total_pts_drv + total_pts_cons, 2),
    "Value Gain (ΣΔPrice)": round(total_gain, 2),
    "Objective (Pts + λ·ΔPrice − Penalty)": round(total_pts_drv + total_pts_cons + LAMBDA_GAIN*total_gain - transfer_pen, 2),
    "Drivers Cost": round(float(best_drivers_df["Cost ($M)"].sum()), 2),
    "Constructors Cost": round(float(best_constructors_df["Cost ($M)"].sum()), 2),
    "Transfers Used": int(transfers_used),
    "Penalty Applied": 0 if (USE_WILDCARD or transfers_used <= FREE_TRANSFERS) else -TRANSFER_PENALTY*(transfers_used - FREE_TRANSFERS),
    "λ (value weight)": LAMBDA_GAIN,
    "Wildcard": USE_WILDCARD,
    "Final Fix": USE_FINAL_FIX
}])

print("=== 🏎️ EQUIPE OTIMIZADA v2 (≤ ${:.1f}M, locks ativos) ===".format(TOTAL_BUDGET))
display(best_drivers_df)
display(best_constructors_df)
display(summary)

# ---------- SQUAD PLANNER ----------
spend_now = float(best_drivers_df["Cost ($M)"].sum() + best_constructors_df["Cost ($M)"].sum())
unspent   = round(TOTAL_BUDGET - spend_now, 2)
est_team_value_after = float(
    spend_now + best_drivers_df["ΔPrice_pred"].fillna(0).sum() + best_constructors_df["ΔPrice_pred"].fillna(0).sum()
)
planner = pd.DataFrame([{
    "Budget Now ($M)": TOTAL_BUDGET,
    "Spend Now ($M)": round(spend_now,2),
    "Unspent Cash ($M)": unspent,
    "Est. Team Value After GP ($M)": round(est_team_value_after,2),
    "Est. Cap Next GP ($M)": round(unspent + best_drivers_df["ΔPrice_pred"].fillna(0).sum() + best_constructors_df["ΔPrice_pred"].fillna(0).sum(), 2)
}])

# ---------- PREPARAR ABAS PARA EXPORT ----------
merged_out = merged.copy()
# Anexa ΔPrice/NewPrice caso existam no drv
for col in ["ΔPrice_pred","NewPrice_est"]:
    if col in drv.columns and col not in merged_out.columns:
        merged_out = merged_out.merge(drv[["Driver", col]], on="Driver", how="left")
merged_out = merged_out.sort_values(PTS_COL, ascending=False)

out_cons = dfc.copy()

drv_mom = None
if {"ΔPrice_pred","NewPrice_est"}.issubset(drv.columns):
    drv_mom = drv[["Driver","Team","CostM","ProjPts","ΔPrice_pred","NewPrice_est"]] \
                .rename(columns={"CostM":"Cost ($M)","ProjPts":"Projected Pts"}) \
                .sort_values("ΔPrice_pred", ascending=False)

cons_mom = None
if "ΔPrice_pred" in dfc.columns:
    cons_mom = dfc[["Constructor","Cost ($M)","ConstructorPts","ΔPrice_pred","NewPrice_est"]] \
                 .sort_values("ΔPrice_pred", ascending=False)

# ---------- EXPORT ÚNICO (um ExcelWriter) ----------
def _save_excel(path):
    with pd.ExcelWriter(path, engine="xlsxwriter") as w:
        merged_out.to_excel(w, sheet_name="Drivers Projections", index=False)
        if 'vepi' in globals():
            vepi.reset_index().to_excel(w, sheet_name="VEPI", index=False)
        out_cons.to_excel(w, sheet_name="Constructors", index=False)
        best_drivers_df.to_excel(w, sheet_name="Best Drivers", index=False)
        best_constructors_df.to_excel(w, sheet_name="Best Constructors", index=False)
        summary.to_excel(w, sheet_name="Best Summary", index=False)
        planner.to_excel(w, sheet_name="Squad Planner", index=False)
        if drv_mom is not None:
            drv_mom.to_excel(w, sheet_name="Price Momentum (Drivers)", index=False)
        if cons_mom is not None:
            cons_mom.to_excel(w, sheet_name="Price Momentum (Cons)", index=False)

try:
    _save_excel(OUT_EXCEL)
    print(f"✅ Excel salvo: {OUT_EXCEL}")
except PermissionError:
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    alt = OUT_EXCEL.replace(".xlsx", f"_{ts}.xlsx")
    _save_excel(alt)
    print(f"⚠️ Arquivo em uso; salvei como: {alt}")
except Exception:
    print("❌ Erro ao salvar Excel:")
    traceback.print_exc()


=== 🏎️ EQUIPE OTIMIZADA v2 (≤ $119.6M, locks ativos) ===


,Driver,Team,Cost ($M),Projected Pts (no mult),Multiplier,Projected Pts (with mult),ΔPrice_pred,NewPrice_est,Value (pts/$M)
0,Oscar Piastri,McLaren,26.0,29.25,2.0,58.50,0.20,26.20,2.250
1,Oliver Bearman,Haas,8.3,8.85,1.0,8.85,0.02,8.32,1.066
2,Nico Hülkenberg,Kick Sauber,6.8,6.25,1.0,6.25,-0.01,6.79,0.919
3,Esteban Ocon,Alpine,6.5,6.20,1.0,6.20,-0.01,6.49,0.954
4,Isack Hadjar,RB,5.9,4.15,1.0,4.15,-0.05,5.85,0.703


,Constructor,Cost ($M),Constructor Pts (Est.),ΔPrice_pred,NewPrice_est
0,McLaren,35.6,74.95,0.22,35.82
1,Red Bull Racing,30.2,35.30,0.04,30.24


,Total Cost ($M),Total Projected Pts,Value Gain (ΣΔPrice),Objective (Pts + λ·ΔPrice − Penalty),Drivers Cost,Constructors Cost,Transfers Used,Penalty Applied,λ (value weight),Wildcard,Final Fix
0,119.3,194.2,0.41,194.32,53.5,65.8,0,0,0.3,False,False


✅ Excel salvo: C:/F1/setup/f1_race_traces_2021/F1_Fantasy_Vegas_Projections_v2_withMomentum.xlsx


In [ ]:
# ===========================================
# SIMULADOR F1 FANTASY – cenários + MC + Final Fix + UI
# ===========================================
import pandas as pd, numpy as np, datetime, traceback, math, re, sys, importlib
from difflib import get_close_matches

def _suggest_name(name, pool_names):
    m = get_close_matches(name, pool_names, n=1, cutoff=0.6)
    return m[0] if m else None


# ---------- Helpers já compatíveis com seu caderno ----------
def _safe_num(s, default=np.nan):
    try:
        return pd.to_numeric(s)
    except Exception:
        return default

def _col_exists(df, col): return (col in df.columns)

# Detecta coluna de pontos projetados do driver, mantendo compatibilidade com seu notebook
DRV_PTS_COL = next((c for c in ["Projected Vegas Pts (no mult)", "Projected Vegas Pts", "Proj Pts (Event)"] if c in merged.columns), None)
if not DRV_PTS_COL:
    raise RuntimeError("Coluna de pontos projetados de pilotos não encontrada em 'merged'.")

# Garante colunas momentum no merged e df_constructors (fallbacks se não houver)
if "ΔPrice_pred" not in merged.columns: merged["ΔPrice_pred"] = 0.0
if "NewPrice_est" not in merged.columns: merged["NewPrice_est"] = np.nan
if "ΔPrice_pred" not in df_constructors.columns: df_constructors["ΔPrice_pred"] = 0.0
if "NewPrice_est" not in df_constructors.columns: df_constructors["NewPrice_est"] = np.nan

# ---------- Busca de registros (case-insensitive, aceita prefixo) ----------
def _find_driver(name: str):
    if not name: return None
    pool = merged.copy()
    # exato
    mask_exact = pool["Driver"].str.casefold() == name.casefold()
    if mask_exact.any(): 
        return pool[mask_exact].iloc[0]
    # prefixo
    mask_prefix = pool["Driver"].str.casefold().str.startswith(name.casefold())
    if mask_prefix.any(): 
        return pool[mask_prefix].iloc[0]
    # tokens (Lando / Piastri)
    tokens = [t for t in re.split(r"\s+", name.strip()) if t]
    if tokens:
        mask_all = pd.Series(True, index=pool.index)
        for t in tokens:
            mask_all &= pool["Driver"].str.casefold().str.contains(t.casefold())
        if mask_all.any(): 
            return pool[mask_all].iloc[0]
    # fuzzy (corrige “vestappen” -> “Verstappen”)
    sug = _suggest_name(name, list(pool["Driver"].unique()))
    if sug:
        return pool[pool["Driver"] == sug].iloc[0]
    return None

def _find_constructor(name: str):
    if not name: return None
    pool = df_constructors.copy()
    mask_exact = pool["Constructor"].str.casefold() == name.casefold()
    if mask_exact.any(): 
        return pool[mask_exact].iloc[0]
    mask_prefix = pool["Constructor"].str.casefold().str.startswith(name.casefold())
    if mask_prefix.any(): 
        return pool[mask_prefix].iloc[0]
    tokens = [t for t in re.split(r"\s+", name.strip()) if t]
    if tokens:
        mask_all = pd.Series(True, index=pool.index)
        for t in tokens:
            mask_all &= pool["Constructor"].str.casefold().str.contains(t.casefold())
        if mask_all.any(): 
            return pool[mask_all].iloc[0]
    sug = _suggest_name(name, list(pool["Constructor"].unique()))
    if sug:
        return pool[pool["Constructor"] == sug].iloc[0]
    return None

# ---------- Contagem de transfers (compatível com Otimizador v2) ----------
def count_transfers(selected_drivers, selected_constructors,
                    current_drivers=None, current_constructors=None):
    current_drivers = current_drivers or []
    current_constructors = current_constructors or []
    if not current_drivers and not current_constructors:
        return 0
    cur_drv, cur_con = set(current_drivers), set(current_constructors)
    sel_drv, sel_con = set(selected_drivers), set(selected_constructors)
    drv_changes = len(cur_drv.symmetric_difference(sel_drv))
    con_changes = len(cur_con.symmetric_difference(sel_con))
    # entrada+saída contam 2, arredondar para cima
    return int(math.ceil(drv_changes/2) + math.ceil(con_changes/2))

# ---------- Núcleo do simulador ----------
def simulate_team(
    drivers, constructors,
    *,
    total_budget: float = 119.6,
    driver_multipliers: dict = None,
    lambda_gain: float = 0.30,
    current_drivers: list = None,
    current_constructors: list = None,
    free_transfers: int = 2,
    transfer_penalty: float = 10,
    use_wildcard: bool = False,
    use_final_fix: bool = False,
    # Extensões:
    scenario: str = "base",           # "base" | "otimista" | "pessimista"
    scenario_margin_pts: float = 0.08,# ±8% nos pontos
    scenario_margin_val: float = 0.10,# ±0.10 no ΔPreço em $M
    mc_iters: int = 0,                # 0 desliga; 2000~5000 leve
    mc_pts_sigma: float = 0.15,       # 15% de desvio relativo em pontos
    mc_val_sigma: float = 0.05,       # 0.05 $M desvio absoluto
    locks: dict = None                # {"drivers":[...], "constructors":[...]} (opcional)
):
    msgs = []
    driver_multipliers = driver_multipliers or {}
    current_drivers = current_drivers or []
    current_constructors = current_constructors or []
    locks = locks or {"drivers": [], "constructors": []}

    # Validação cardinalidade
    if len(drivers) != 5:
        msgs.append(f"Necessário informar **5** pilotos (recebido {len(drivers)}).")
    if len(constructors) != 2:
        msgs.append(f"Necessário informar **2** construtores (recebido {len(constructors)}).")

    # Remover vazios / espaços
    drivers = [s for s in (t.strip() for t in drivers) if s]
    constructors = [s for s in (t.strip() for t in constructors) if s]

    # Busca registros
    drv_rows = []
    not_found = []
    for name in drivers:
        row = _find_driver(name)
        if row is None: not_found.append(name)
        else: drv_rows.append(row)
    con_rows = []
    for name in constructors:
        row = _find_constructor(name)
        if row is None: not_found.append(name)
        else: con_rows.append(row)

    if not_found:
        msgs.append("Não encontrados: " + ", ".join(not_found))

    # Duplicatas
    if len(set(drivers)) != len(drivers):
        msgs.append("Pilotos duplicados na lista.")
    if len(set(constructors)) != len(constructors):
        msgs.append("Construtores duplicados na lista.")

    # Locks (se fornecidos)
    if locks.get("drivers"):
        missing = [x for x in locks["drivers"] if x not in drivers]
        if missing:
            msgs.append("Lock de piloto não atendido: " + ", ".join(missing))
    if locks.get("constructors"):
        missing = [x for x in locks["constructors"] if x not in constructors]
        if missing:
            msgs.append("Lock de construtor não atendido: " + ", ".join(missing))

    if msgs:
        return {"drivers_df": pd.DataFrame(), "constructors_df": pd.DataFrame(),
                "summary_df": pd.DataFrame(), "feasible": False, "messages": msgs}

    # Monta DF de pilotos
    dlist = []
    for r in drv_rows:
        cost = _safe_num(r.get("Cost ($M)"), 0.0)
        proj = _safe_num(r.get(DRV_PTS_COL), 0.0)
        mult = float(driver_multipliers.get(r["Driver"], 1.0))
        dprice = _safe_num(r.get("ΔPrice_pred"), 0.0)
        newp   = _safe_num(r.get("NewPrice_est"), np.nan)
        dlist.append({
            "Piloto": r["Driver"],
            "Equipe": r.get("Team", ""),
            "Custo ($M)": float(cost),
            "Proj. (sem mult)": float(proj),
            "Multiplicador": mult,
            "Proj. (com mult)": float(proj * mult),
            "ΔPreço (previsto)": float(dprice),
            "Preço pós-GP (prev.)": float(newp) if pd.notna(newp) else float(cost + dprice),
            "Valor (pts/$M)": (proj * mult / cost) if cost else np.nan
        })
    drivers_df = pd.DataFrame(dlist).sort_values("Proj. (com mult)", ascending=False).reset_index(drop=True)

    # Construtores
    clist = []
    for r in con_rows:
        cost = _safe_num(r.get("Cost ($M)"), 0.0)
        cpts = _safe_num(r.get("ConstructorPts") if "ConstructorPts" in r.index else r.get("Constructor Pts (Est.)"), 0.0)
        dprice = _safe_num(r.get("ΔPrice_pred"), 0.0)
        newp   = _safe_num(r.get("NewPrice_est"), np.nan)
        clist.append({
            "Construtor": r["Constructor"],
            "Custo ($M)": float(cost),
            "Pontos do Construtor": float(cpts),
            "ΔPreço (previsto)": float(dprice),
            "Preço pós-GP (prev.)": float(newp) if pd.notna(newp) else float(cost + dprice)
        })
    constructors_df = pd.DataFrame(clist).sort_values("Pontos do Construtor", ascending=False).reset_index(drop=True)

    # Custos e pontos base
    cost_total = float(drivers_df["Custo ($M)"].sum() + constructors_df["Custo ($M)"].sum())
    pts_total  = float(drivers_df["Proj. (com mult)"].sum() + constructors_df["Pontos do Construtor"].sum())
    dprice_sum = float(drivers_df["ΔPreço (previsto)"].sum() + constructors_df["ΔPreço (previsto)"].sum())

    # Cenários determinísticos
    if scenario in ("otimista","pessimista"):
        sgn = +1 if scenario=="otimista" else -1
        pts_total *= (1 + sgn*scenario_margin_pts)
        dprice_sum += sgn*scenario_margin_val

    # Penalidade de transfers
    transfers = count_transfers([*drivers_df["Piloto"]], [*constructors_df["Construtor"]],
                                current_drivers=current_drivers, current_constructors=current_constructors)
    extra = max(0, transfers - free_transfers)
    penalty = 0 if use_wildcard else transfer_penalty * extra

    # Objetivo híbrido
    objective = pts_total + lambda_gain * dprice_sum - penalty

    # MC leve (opcional)
    mc_result = {}
    if mc_iters and mc_iters > 0:
        rng = np.random.default_rng(42)
        # Vetores base
        drv_pts_base = drivers_df["Proj. (com mult)"].to_numpy()
        cons_pts_base = constructors_df["Pontos do Construtor"].to_numpy()
        drv_val_base = drivers_df["ΔPreço (previsto)"].to_numpy()
        cons_val_base = constructors_df["ΔPreço (previsto)"].to_numpy()

        pts_samples = []
        val_samples = []
        obj_samples = []
        for _ in range(int(mc_iters)):
            drv_pts = drv_pts_base * (1.0 + rng.normal(0, mc_pts_sigma, size=drv_pts_base.shape))
            cons_pts = cons_pts_base * (1.0 + rng.normal(0, mc_pts_sigma, size=cons_pts_base.shape))
            drv_val = drv_val_base + rng.normal(0, mc_val_sigma, size=drv_val_base.shape)
            cons_val = cons_val_base + rng.normal(0, mc_val_sigma, size=cons_val_base.shape)

            pts = float(drv_pts.sum() + cons_pts.sum())
            val = float(drv_val.sum() + cons_val.sum())
            obj = pts + lambda_gain * val - penalty
            pts_samples.append(pts); val_samples.append(val); obj_samples.append(obj)

        def _q(a, q): 
            i = int(np.clip(round(q*(len(a)-1)),0,len(a)-1)); 
            return float(np.sort(a)[i])

        mc_result = {
            "Pts p05": _q(pts_samples, 0.05), "Pts p50": _q(pts_samples, 0.50), "Pts p95": _q(pts_samples, 0.95),
            "ΔPreço p05": _q(val_samples, 0.05), "ΔPreço p50": _q(val_samples, 0.50), "ΔPreço p95": _q(val_samples, 0.95),
            "Obj p05": _q(obj_samples, 0.05), "Obj p50": _q(obj_samples, 0.50), "Obj p95": _q(obj_samples, 0.95),
        }

    # Heurística de Final Fix (1 troca de piloto que mais aumenta o objetivo)
    final_fix_suggestion = None
    if use_final_fix:
        # Candidato: substituir o pior piloto por outro do pool que caiba no cap e suba o objetivo
        pool = merged.copy()
        pool = pool[~pool["Driver"].isin(drivers_df["Piloto"])]  # não repetir
        # ordena por ProjPts + λ*ΔPreço simples (sem mult); mult será aplicado
        score = _safe_num(pool[DRV_PTS_COL], 0) + lambda_gain * _safe_num(pool["ΔPrice_pred"], 0)
        pool = pool.assign(_score=score).sort_values("_score", ascending=False)
        worst_idx = drivers_df["Proj. (com mult)"].idxmin()
        worst = drivers_df.loc[worst_idx]
        budget_left = total_budget - cost_total + worst["Custo ($M)"]

        for _, cand in pool.iterrows():
            c_cost = float(_safe_num(cand.get("Cost ($M)"), 0.0))
            if c_cost <= budget_left:
                mult = float(driver_multipliers.get(cand["Driver"], 1.0))
                c_pts = float(_safe_num(cand.get(DRV_PTS_COL), 0.0)) * mult
                c_val = float(_safe_num(cand.get("ΔPrice_pred"), 0.0))
                gain_pts = c_pts - worst["Proj. (com mult)"]
                gain_val = c_val - worst["ΔPreço (previsto)"]
                delta_obj = gain_pts + lambda_gain * gain_val
                if delta_obj > 0:
                    final_fix_suggestion = {
                        "Out": worst["Piloto"], "In": cand["Driver"],
                        "ΔObjetivo": round(delta_obj, 2),
                        "Custo In ($M)": round(c_cost,2),
                        "Pts In (com mult)": round(c_pts,2),
                        "ΔPreço In": round(c_val,2)
                    }
                break

    feasible = cost_total <= total_budget
    cash_left = round(total_budget - cost_total, 2)
    cap_next  = round(cash_left + dprice_sum, 2)

    # Summary DF
    row = {
        "Custo Total ($M)": round(cost_total, 2),
        "Total de Pontos (proj.)": round(pts_total, 2),
        "Ganho de Valor (ΣΔPreço)": round(dprice_sum, 2),
        "Objetivo (Pts + λ·ΔPreço − Penalidade)": round(objective, 2),
        "Transfers usados": int(transfers),
        "Penalidade aplicada": 0 if use_wildcard else -transfer_penalty*max(0, extra),
        "λ (peso de valor)": lambda_gain,
        "Budget Agora ($M)": total_budget,
        "Gasto Agora ($M)": round(cost_total, 2),
        "Caixa Livre ($M)": cash_left,
        "Cap Estimado Próximo GP ($M)": cap_next,
        "Cenário": scenario,
        "MC iters": int(mc_iters)
    }
    row.update(mc_result)
    summary_df = pd.DataFrame([row])

    # Equipe unificada (para export fácil)
    eq_cols = ["Tipo","Nome","Equipe/Construtor","Custo ($M)","Pts (proj.)","ΔPreço (previsto)","Preço pós-GP (prev.)"]
    eq_rows = []
    for _, r in drivers_df.iterrows():
        eq_rows.append(["Piloto", r["Piloto"], r["Equipe"], r["Custo ($M)"], r["Proj. (com mult)"], r["ΔPreço (previsto)"], r["Preço pós-GP (prev.)"]])
    for _, r in constructors_df.iterrows():
        eq_rows.append(["Construtor", r["Construtor"], r["Construtor"], r["Custo ($M)"], r["Pontos do Construtor"], r["ΔPreço (previsto)"], r["Preço pós-GP (prev.)"]])
    team_df = pd.DataFrame(eq_rows, columns=eq_cols)

    return {
        "drivers_df": drivers_df,
        "constructors_df": constructors_df,
        "summary_df": summary_df,
        "feasible": feasible,
        "messages": msgs,
        "team_df": team_df,
        "final_fix": final_fix_suggestion
    }

# ---------- Renderização e export para a MESMA planilha (regera tudo + adiciona simulação) ----------
def export_with_simulation(_sim, sheet_name=None):
    """Reescreve OUT_EXCEL completo (Drivers Projections, VEPI, Constructors, Best*, Planner, Momentum, Equipe Ideal)
       e inclui uma aba de Simulação adicional."""
    try:
        from xlsxwriter import Workbook  # só para garantir instalação
    except Exception:
        raise RuntimeError("xlsxwriter não encontrado. Instale com: pip install xlsxwriter")

    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M")
    sheet_name = sheet_name or f"Simulação – {ts}"

    # Usa arte/estilo do seu export “bonitão” se variáveis estiverem presentes; caso não, cria arquivo novo básico.
    try:
        # Reaproveita variáveis calculadas pelo Otimizador v2:
        _merged_out = merged.copy()
        if "ΔPrice_pred" in merged.columns and "NewPrice_est" in merged.columns:
            pass  # já estão
        # Construtores “out_cons”: use df_constructors atual
        _out_cons = df_constructors.copy()

        # Tabelas “Best*” já devem existir do v2; se não existirem, cria placeholders
        _best_drivers = globals().get("best_drivers_df", pd.DataFrame())
        _best_cons    = globals().get("best_constructors_df", pd.DataFrame())
        _summary      = globals().get("summary", pd.DataFrame())
        _planner      = globals().get("planner", pd.DataFrame())
        _drv_mom      = globals().get("drv_mom", None)
        _cons_mom     = globals().get("cons_mom", None)
        _vepi         = globals().get("vepi", None)

        # Estilizado (versão leve aqui para caber numa célula)
        with pd.ExcelWriter(OUT_EXCEL, engine="xlsxwriter") as writer:
            wb = writer.book
            h1 = wb.add_format({"bold":True,"font_color":"white","bg_color":"#0F6CBD","align":"center","valign":"vcenter","border":1})
            h2 = wb.add_format({"bold":True,"font_color":"white","bg_color":"#1F2937","align":"center","valign":"vcenter","border":1})
            m2 = wb.add_format({"num_format":"#,##0.00","border":1})
            m3 = wb.add_format({"num_format":"#,##0.000","border":1})
            mo = wb.add_format({"num_format":"#,##0.00","border":1})
            tx = wb.add_format({"border":1})
            tl = wb.add_format({"bold":True,"bg_color":"#E5F0FF","border":1,"num_format":"#,##0.00"})

            # Drivers Projections PT-BR (colunas usuais)
            drivers_pt = _merged_out.rename(columns={
                "Driver":"Piloto","Team":"Equipe","Cost ($M)":"Custo ($M)",
                "Avg Pts/GP":"Pontos médios/GP","Pts per $M":"Pontos por $M",
                "Proj Pts (Sprint wknd)":"Proj. (fim de semana Sprint)",
                "Value (Sprint pts per $M)":"Valor (Sprint pts/$M)",
                "Proj Pts (Event)":"Proj. (Evento)",
                "VEPI_adj":"VEPI (ajustado)",
                DRV_PTS_COL:"Proj. Vegas (sem multiplicador)",
                "ΔPrice_pred":"ΔPreço (previsto)","NewPrice_est":"Preço pós-GP (prev.)"
            })
            order_cols = ["Piloto","Custo ($M)","Pontos médios/GP","Pontos por $M","Proj. (fim de semana Sprint)","Valor (Sprint pts/$M)","Proj. (Evento)","VEPI (ajustado)","Proj. Vegas (sem multiplicador)","Equipe","ΔPreço (previsto)","Preço pós-GP (prev.)"]
            drivers_pt = drivers_pt[[c for c in order_cols if c in drivers_pt.columns]]
            drivers_pt.sort_values(by=[c for c in ["Proj. Vegas (sem multiplicador)"] if c in drivers_pt.columns], ascending=False).to_excel(writer, sheet_name="Drivers Projections", index=False)
            ws = writer.sheets["Drivers Projections"]
            for i,c in enumerate(drivers_pt.columns): ws.write(0,i,c,h1)

            # VEPI
            if _vepi is not None and not _vepi.empty:
                _vepi.reset_index().rename(columns={"driver":"Piloto","VEPI":"VEPI"}).to_excel(writer, sheet_name="VEPI", index=False)
                wv = writer.sheets["VEPI"]; wv.write(0,0,"Piloto",h1); wv.write(0,1,"VEPI",h1)

            # Constructors
            cons_pt = _out_cons.rename(columns={
                "Constructor":"Construtor","Cost ($M)":"Custo ($M)",
                "ConstructorPts":"Pontos do Construtor",
                "Constructor Pts (Est.)":"Pontos do Construtor",
                "ΔPrice_pred":"ΔPreço (previsto)","NewPrice_est":"Preço pós-GP (prev.)"
            })
            cons_pt.to_excel(writer, sheet_name="Constructors", index=False)
            wc = writer.sheets["Constructors"]
            for i,c in enumerate(cons_pt.columns): wc.write(0,i,c,h1)

            # Best*
            if not _best_drivers.empty:
                _best_drivers.to_excel(writer, sheet_name="Best Drivers", index=False)
                wbd = writer.sheets["Best Drivers"]; 
                for i,c in enumerate(_best_drivers.columns): wbd.write(0,i,c,h1)
            if not _best_cons.empty:
                _best_cons.to_excel(writer, sheet_name="Best Constructors", index=False)
                wbc = writer.sheets["Best Constructors"]; 
                for i,c in enumerate(_best_cons.columns): wbc.write(0,i,c,h1)
            if not _summary.empty:
                _summary.to_excel(writer, sheet_name="Best Summary", index=False)
                wbs = writer.sheets["Best Summary"]; 
                for i,c in enumerate(_summary.columns): wbs.write(0,i,c,h1)
            if not _planner.empty:
                _planner.to_excel(writer, sheet_name="Squad Planner", index=False)
                wpl = writer.sheets["Squad Planner"]; 
                for i,c in enumerate(_planner.columns): wpl.write(0,i,c,h1)
            if _drv_mom is not None and not isinstance(_drv_mom, int):
                _drv_mom.to_excel(writer, sheet_name="Price Momentum (Drivers)", index=False)
                wmd = writer.sheets["Price Momentum (Drivers)"]; 
                for i,c in enumerate(_drv_mom.columns): wmd.write(0,i,c,h1)
            if _cons_mom is not None and not isinstance(_cons_mom, int):
                _cons_mom.to_excel(writer, sheet_name="Price Momentum (Cons)", index=False)
                wmc = writer.sheets["Price Momentum (Cons)"]; 
                for i,c in enumerate(_cons_mom.columns): wmc.write(0,i,c,h1)

            # Equipe Ideal (se existir no contexto)
            if "best_drivers_df" in globals() and "best_constructors_df" in globals():
                # montar totalizador rapidamente
                eq_cols = ["Tipo","Nome","Equipe/Construtor","Custo ($M)","Pts (proj.)","ΔPreço (previsto)","Preço pós-GP (prev.)"]
                eq_rows = []
                for _, r in best_drivers_df.iterrows():
                    eq_rows.append(["Piloto", r["Driver"], r.get("Team",""), r["Cost ($M)"], r["Projected Pts (with mult)"], r.get("ΔPrice_pred",0.0), r.get("NewPrice_est", np.nan)])
                for _, r in best_constructors_df.iterrows():
                    eq_rows.append(["Construtor", r["Constructor"], r["Constructor"], r["Cost ($M)"], r["Constructor Pts (Est.)"], r.get("ΔPrice_pred",0.0), r.get("NewPrice_est", np.nan)])
                eq_df = pd.DataFrame(eq_rows, columns=eq_cols)
                eq_df.to_excel(writer, sheet_name="Equipe Ideal", index=False)
                wi = writer.sheets["Equipe Ideal"]
                for i,c in enumerate(eq_cols): wi.write(0,i,c,h2)
                last = len(eq_df)+1
                wi.write(last,0,"Totais", tl)
                wi.write_formula(last,3,f"=SUM(D2:D{last})", tl)
                wi.write_formula(last,4,f"=SUM(E2:E{last})", tl)
                wi.write_formula(last,5,f"=SUM(F2:F{last})", tl)
                wi.write_formula(last,6,f"=SUM(G2:G{last})", tl)

            # Nova aba: Simulação
            _sim["team_df"].to_excel(writer, sheet_name=sheet_name, index=False)
            ws = writer.sheets[sheet_name]
            for i,c in enumerate(_sim["team_df"].columns): ws.write(0,i,c,h2)
            # totalizador
            last = len(_sim["team_df"])+1
            ws.write(last,0,"Totais", tl)
            ws.write_formula(last,3,f"=SUM(D2:D{last})", tl)
            ws.write_formula(last,4,f"=SUM(E2:E{last})", tl)
            ws.write_formula(last,5,f"=SUM(F2:F{last})", tl)
            ws.write_formula(last,6,f"=SUM(G2:G{last})", tl)

            # Quadro Resumo ao lado (col I)
            summary = _sim["summary_df"]
            ws.write(1, 9, "Resumo da Simulação", wb.add_format({"bold":True,"font_color":"white","bg_color":"#2563EB","align":"center","valign":"vcenter"}))
            r = 3
            for col in summary.columns:
                ws.write(r, 8, col, tx)
                val = summary.iloc[0][col]
                ws.write(r, 9, float(val) if pd.api.types.is_number(val) else val, m2 if isinstance(val,(int,float,np.floating)) else tx)
                r += 1
            # Final Fix suggestion (se houver)
            if _sim.get("final_fix"):
                ws.write(r+1, 8, "Sugestão Final Fix", tx)
                ff = _sim["final_fix"]
                ws.write(r+2, 8, f"Out: {ff['Out']}", tx)
                ws.write(r+3, 8, f"In: {ff['In']} (Custo {ff['Custo In ($M)']})", tx)
                ws.write(r+4, 8, f"ΔObjetivo: {ff['ΔObjetivo']}", tx)

        print(f"✅ Simulação exportada em: {OUT_EXCEL} (aba: {sheet_name})")
    except PermissionError:
        alt = OUT_EXCEL.replace(".xlsx", f"_sim_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx")
        with pd.ExcelWriter(alt, engine="xlsxwriter") as writer:
            _sim["team_df"].to_excel(writer, sheet_name="Simulação", index=False)
            _sim["summary_df"].to_excel(writer, sheet_name="Resumo", index=False)
        print(f"⚠️ OUT_EXCEL em uso; salvei simulação em: {alt}")
    except Exception:
        print("❌ Erro na exportação da simulação:")
        traceback.print_exc()

def render_simulation(sim, export: bool = True, sheet_name: str = None):
    from IPython.display import display, HTML
    if sim["messages"]:
        display(HTML("<b>Mensagens:</b> " + "<br>".join(sim["messages"])))
    display(sim["drivers_df"]); display(sim["constructors_df"]); display(sim["summary_df"])
    if export:
        export_with_simulation(sim, sheet_name=sheet_name)

# ---------- UI interativa (ipywidgets) ----------
def _has_ipywidgets():
    try:
        import ipywidgets as widgets  # noqa
        return True
    except Exception:
        return False

if _has_ipywidgets():
    import ipywidgets as widgets
    from IPython.display import display, HTML

    # Campos básicos
    inp_drivers = widgets.Text(
        value="Oscar Piastri, Lando Norris, George Russell, Charles Leclerc, Oliver Bearman",
        description="Pilotos (5):", layout=widgets.Layout(width="100%")
    )
    inp_cons = widgets.Text(
        value="McLaren, Red Bull Racing",
        description="Construtores (2):", layout=widgets.Layout(width="100%")
    )
    inp_budget = widgets.FloatText(value=119.6, description="💰 Orçamento:", step=0.1)
    inp_lambda = widgets.FloatSlider(value=0.30, min=0.10, max=0.60, step=0.05, description="λ valorização")
    inp_free = widgets.IntSlider(value=2, min=0, max=6, step=1, description="Free transfers")
    inp_pen = widgets.FloatText(value=10.0, description="Penalidade/transfer", step=1.0)
    chk_wc = widgets.Checkbox(value=False, description="Wildcard")
    chk_ff = widgets.Checkbox(value=False, description="Final Fix (pós-quali)")
    dd_scenario = widgets.Dropdown(options=["base","otimista","pessimista"], value="base", description="Cenário")
    iters_mc = widgets.IntSlider(value=0, min=0, max=5000, step=500, description="MC iterações")
    sigma_pts = widgets.FloatSlider(value=0.15, min=0.05, max=0.35, step=0.01, description="σ pontos (rel.)")
    sigma_val = widgets.FloatSlider(value=0.05, min=0.00, max=0.20, step=0.01, description="σ ΔPreço ($M)")
    txt_mult = widgets.Text(value="Oscar Piastri:2.0", description="Multiplicadores")
    txt_cur_drv = widgets.Text(value="", description="Time atual - pilotos")
    txt_cur_con = widgets.Text(value="", description="Time atual - constr.")
    btn = widgets.Button(description="Simular e Exportar", button_style="primary")

    def _parse_list(s):
        return [x.strip() for x in s.split(",") if x.strip()]

    def _parse_mult(s):
        out = {}
        for part in [p.strip() for p in s.split(",") if p.strip()]:
            if ":" in part:
                k,v = part.split(":",1)
                try: out[k.strip()] = float(v)
                except: pass
        return out

    def on_click(_):
        drivers = _parse_list(inp_drivers.value)
        constructors = _parse_list(inp_cons.value)
        driver_mult = _parse_mult(txt_mult.value)
        cur_drv = _parse_list(txt_cur_drv.value)
        cur_con = _parse_list(txt_cur_con.value)

        sim = simulate_team(
            drivers, constructors,
            total_budget=inp_budget.value,
            driver_multipliers=driver_mult,
            lambda_gain=inp_lambda.value,
            current_drivers=cur_drv,
            current_constructors=cur_con,
            free_transfers=inp_free.value,
            transfer_penalty=inp_pen.value,
            use_wildcard=chk_wc.value,
            use_final_fix=chk_ff.value,
            scenario=dd_scenario.value,
            mc_iters=iters_mc.value,
            mc_pts_sigma=sigma_pts.value,
            mc_val_sigma=sigma_val.value
        )
        sheet = f"Simulação – {datetime.datetime.now().strftime('%Y%m%d_%H%M')}"
        render_simulation(sim, export=True, sheet_name=sheet)

    btn.on_click(on_click)

    ui = widgets.VBox([
        widgets.HTML("<h3>Simulador F1 Fantasy – selecione 5 pilotos e 2 construtores</h3>"),
        inp_drivers, inp_cons,
        widgets.HBox([inp_budget, inp_lambda, dd_scenario]),
        widgets.HBox([iters_mc, sigma_pts, sigma_val]),
        widgets.HBox([inp_free, inp_pen, chk_wc, chk_ff]),
        txt_mult,
        widgets.HBox([txt_cur_drv, txt_cur_con]),
        btn
    ])
    display(ui)
else:
    print("ℹ️ ipywidgets não disponível. Use a função simulate_team(...) manualmente, por exemplo:\n"
          "sim = simulate_team(['Oscar Piastri','Lando Norris','George Russell','Charles Leclerc','Oliver Bearman'],\n"
          "                     ['McLaren','Red Bull Racing'], total_budget=119.6,\n"
          "                     driver_multipliers={'Oscar Piastri':2.0}, lambda_gain=0.30,\n"
          "                     current_drivers=[], current_constructors=[], free_transfers=2,\n"
          "                     transfer_penalty=10, use_wildcard=False, use_final_fix=False,\n"
          "                     scenario='base', mc_iters=0)\n"
          "render_simulation(sim, export=True)")


,Piloto,Equipe,Custo ($M),Proj. (sem mult),Multiplicador,Proj. (com mult),ΔPreço (previsto),Preço pós-GP (prev.),Valor (pts/$M)
0,Oscar Piastri,McLaren,26.0,29.25,2.0,58.50,0.0,26.0,2.250000
1,Lando Norris,McLaren,30.4,30.70,1.0,30.70,0.0,30.4,1.009868
2,George Russell,Mercedes,23.4,23.85,1.0,23.85,0.0,23.4,1.019231
3,Charles Leclerc,Ferrari,23.3,17.30,1.0,17.30,0.0,23.3,0.742489
4,Oliver Bearman,Haas,8.3,8.85,1.0,8.85,0.0,8.3,1.066265


,Construtor,Custo ($M),Pontos do Construtor,ΔPreço (previsto),Preço pós-GP (prev.)
0,McLaren,35.6,74.95,0.0,35.6
1,Red Bull Racing,30.2,35.30,0.0,30.2


,Custo Total ($M),Total de Pontos (proj.),Ganho de Valor (ΣΔPreço),Objetivo (Pts + λ·ΔPreço − Penalidade),Transfers usados,Penalidade aplicada,λ (peso de valor),Budget Agora ($M),Gasto Agora ($M),Caixa Livre ($M),Cap Estimado Próximo GP ($M),Cenário,MC iters
0,177.2,249.45,0.0,249.45,0,-0.0,0.3,119.6,177.2,-57.6,-57.6,base,0


,Piloto,Equipe,Custo ($M),Proj. (sem mult),Multiplicador,Proj. (com mult),ΔPreço (previsto),Preço pós-GP (prev.),Valor (pts/$M)
0,Oscar Piastri,McLaren,26.0,29.25,2.0,58.50,0.0,26.0,2.250000
1,Lando Norris,McLaren,30.4,30.70,1.0,30.70,0.0,30.4,1.009868
2,Charles Leclerc,Ferrari,23.3,17.30,1.0,17.30,0.0,23.3,0.742489
3,Oliver Bearman,Haas,8.3,8.85,1.0,8.85,0.0,8.3,1.066265
4,Gabriel Bortoleto,???,7.3,3.85,1.0,3.85,0.0,7.3,0.527397


,Construtor,Custo ($M),Pontos do Construtor,ΔPreço (previsto),Preço pós-GP (prev.)
0,McLaren,35.6,74.95,0.0,35.6
1,Red Bull Racing,30.2,35.30,0.0,30.2


,Custo Total ($M),Total de Pontos (proj.),Ganho de Valor (ΣΔPreço),Objetivo (Pts + λ·ΔPreço − Penalidade),Transfers usados,Penalidade aplicada,λ (peso de valor),Budget Agora ($M),Gasto Agora ($M),Caixa Livre ($M),Cap Estimado Próximo GP ($M),Cenário,MC iters
0,161.1,229.45,0.0,229.45,0,-0.0,0.3,100.0,161.1,-61.1,-61.1,base,0
